In [2]:
import cv2
import numpy as np
import mediapipe as mp
import os

In [3]:
import cv2
import numpy as np
import mediapipe as mp
import os

def image_to_minimal_line_drawing(image_path, output_path):
    """
    Converts an image of a person to a highly minimal, thick-lined drawing 
    by adjusting blur, Canny thresholds, and applying dilation.
    """
    # 1. Initialize MediaPipe Selfie Segmentation
    mp_selfie_segmentation = mp.solutions.selfie_segmentation
    
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as segmentor:
        
        # Read the image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not read image at {image_path}")
            return

        # --- Background Removal (Same as before) ---
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb.flags.writeable = False
        results = segmentor.process(img_rgb)
        img_rgb.flags.writeable = True

        if results.segmentation_mask is None:
            print("Warning: No person detected. Processing entire image...")
            processed_img = img 
        else:
            threshold = 0.1
            binary_mask = results.segmentation_mask > threshold
            mask_3channel = np.stack((binary_mask,) * 3, axis=-1)
            
            # Use white background for contrast
            background_color = [255, 255, 255] 
            solid_background = np.full(img.shape, background_color, dtype=np.uint8)
            processed_img = np.where(mask_3channel, img, solid_background)
        
        # --- Edge Detection with Minimal Settings ---
        
        gray_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)

        # 1. STRONGER GAUSSIAN BLUR (Increased kernel size from 5 to 11)
        # Larger kernel simplifies features, reducing small, noisy lines.
        blurred_img = cv2.GaussianBlur(gray_img, (11, 11), 0)

        # 2. TIGHTER CANNY THRESHOLDS (Increased both high and low values)
        # Only the absolute strongest edges will be kept.
        low_threshold = 100
        high_threshold = 200 
        edges = cv2.Canny(blurred_img, low_threshold, high_threshold)

        # 3. POST-PROCESSING: DILATION to Thicken and Connect Lines
        # Dilation expands the white pixels (edges), making lines thicker and bridging small gaps.
        kernel = np.ones((3, 3), np.uint8) # Defines the size/shape of the thickening area
        dilated_edges = cv2.dilate(edges, kernel, iterations=1) # Increase iterations for max thickness

        # Invert the edges to get black lines on a white background
        line_drawing = cv2.bitwise_not(dilated_edges)

        # Save the result
        cv2.imwrite(output_path, line_drawing)
        print(f"Minimal line drawing saved to: {output_path}")

# Example Usage:
# Note: You need an image named 'person_image.jpg' in the same directory.
# image_to_line_drawing_with_segmentation('person_image.jpg', 'line_drawing_output.png')

In [17]:
image_to_minimal_line_drawing('divija.png', 'line_drawing_output_minimal.png')

I0000 00:00:1760825571.235488 48011750 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760825571.237021 48018601 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Minimal line drawing saved to: line_drawing_output_minimal.png


In [24]:
def image_to_minimal_line_drawing_fixed(image_path, output_path):
    """
    Adjusts the blur and Canny thresholds to create minimal lines while 
    retaining essential facial features.
    """
    # 1. Initialize MediaPipe Selfie Segmentation
    mp_selfie_segmentation = mp.solutions.selfie_segmentation
    
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as segmentor:
        
        # Read the image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not read image at {image_path}")
            return

        # --- Background Removal --- (Using the same method)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb.flags.writeable = False
        results = segmentor.process(img_rgb)
        img_rgb.flags.writeable = True

        if results.segmentation_mask is None:
            processed_img = img 
        else:
            threshold = 0.1
            binary_mask = results.segmentation_mask > threshold
            mask_3channel = np.stack((binary_mask,) * 3, axis=-1)
            background_color = [255, 255, 255] 
            solid_background = np.full(img.shape, background_color, dtype=np.uint8)
            processed_img = np.where(mask_3channel, img, solid_background)
        
        # --- Edge Detection with Detail Preservation ---
        
        gray_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
        high_contrast_img = cv2.equalizeHist(gray_img)
        # 1. MILDER GAUSSIAN BLUR (Reduced kernel size from 11 to 7 or 9)
        # Less blur preserves finer gradients needed for facial features.
        blurred_img = cv2.GaussianBlur(high_contrast_img, (11, 11), 0) # Try (9, 9) if features are still too faint

        # 2. LOOSER CANNY THRESHOLDS (Lowered thresholds slightly)
        # This allows slightly weaker gradients (like those for a mouth or eyebrow) to be detected.
        # Original minimal: (100, 200) -> New balanced: (80, 180)
        low_threshold = 20
        high_threshold = 180 
        edges = cv2.Canny(blurred_img, low_threshold, high_threshold)

        # 3. POST-PROCESSING: DILATION to Thicken and Connect Lines
        # Dilation is kept to maintain the desired thick, continuous look.
        kernel = np.ones((3, 3), np.uint8) 
        dilated_edges = cv2.dilate(edges, kernel, iterations=1) 

        # Invert to get black lines on a white background
        line_drawing = cv2.bitwise_not(dilated_edges)

        # Save the result
        cv2.imwrite(output_path, blurred_img)
        print(f"Minimal line drawing with features saved to: {output_path}")

In [25]:
image_to_minimal_line_drawing_fixed('divija.png', 'line_drawing_output_minimal_fixed.png')

I0000 00:00:1760825745.588507 48011750 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760825745.589607 48021489 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Minimal line drawing with features saved to: line_drawing_output_minimal_fixed.png


In [17]:
def image_to_minimal_line_drawing_final(image_path, output_path):
    """
    Uses CLAHE (Adaptive Contrast) to boost facial features, then applies a strong 
    blur and minimal Canny settings for a highly stylized, clean line drawing.
    """
    # 1. Initialize MediaPipe Selfie Segmentation
    mp_selfie_segmentation = mp.solutions.selfie_segmentation
    
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as segmentor:
        
        # Read the image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not read image at {image_path}")
            return

        # --- Background Removal ---
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb.flags.writeable = False
        results = segmentor.process(img_rgb)
        img_rgb.flags.writeable = True

        processed_img = img 
        if results.segmentation_mask is not None:
            threshold = 0.1
            binary_mask = results.segmentation_mask > threshold
            mask_3channel = np.stack((binary_mask,) * 3, axis=-1)
            background_color = [255, 255, 255] 
            solid_background = np.full(img.shape, background_color, dtype=np.uint8)
            processed_img = np.where(mask_3channel, img, solid_background)
        
        # --- Edge Detection with CLAHE Contrast Enhancement ---
        
        gray_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
        
        # 1. ADAPTIVE CONTRAST (CLAHE): Aggressively boost local features
        # Increased clipLimit (e.g., to 3.0) for stronger detail enhancement 
        # (simulates your "making details more clearer" setting).
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        high_contrast_img = clahe.apply(gray_img) 
        
        # 2. STRONG GAUSSIAN BLUR (Increased kernel size to eliminate insignificant stuff)
        # This acts like your "washing out insignificant stuff" setting.
        # It removes noise and minor texture, but the high-contrast features survive.
        blurred_img = cv2.GaussianBlur(high_contrast_img, (13, 13), 0)

        # 3. TIGHT CANNY THRESHOLDS (Only grab the strongest, essential lines)
        low_threshold = 100
        high_threshold = 200 
        edges = cv2.Canny(blurred_img, 0, 100)

        # 4. DILATION to Thicken and Connect Lines
        kernel = np.ones((5, 5), np.uint8) 
        dilated_edges = cv2.dilate(edges, kernel, iterations=2) 

        # Invert to get black lines on a white background
        line_drawing = cv2.bitwise_not(dilated_edges)

        # Save the result
        cv2.imwrite(output_path, line_drawing)
        print(f"Final minimal line drawing saved to: {output_path}")

# Example Usage:
image_to_minimal_line_drawing_final('divija.png', 'minimal_line_drawing_final.png')

I0000 00:00:1760839269.756599 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760839269.758616 48142685 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Final minimal line drawing saved to: minimal_line_drawing_final.png


In [81]:
# --- NEW MINIMAL LINE DRAWING WITH MEDIAPIPE AND SELECTIVE CLOTHING BLUR ---
def image_to_minimal_line_drawing_final(image_path, output_path):
    """
    Uses MediaPipe Selfie Segmentation to remove the background, 
    then applies CLAHE (Adaptive Contrast) to boost facial features, 
    followed by strong blur and minimal Canny settings for a highly 
    stylized, clean line drawing.
    """
    print(f"Starting MediaPipe/CLAHE line drawing for: {image_path}")
    
    # 1. Initialize MediaPipe Selfie Segmentation
    mp_selfie_segmentation = mp.solutions.selfie_segmentation
    
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as segmentor:
        
        # Read the image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not read image at {image_path}")
            return

        # --- Background Removal using MediaPipe ---
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb.flags.writeable = False
        # MediaPipe requires an RGB input
        results = segmentor.process(img_rgb) 
        img_rgb.flags.writeable = True

        processed_img = img 
        if results.segmentation_mask is not None:
            # Create a mask for the foreground (person)
            threshold = 0.1
            binary_mask = results.segmentation_mask > threshold
            # Stack the 1-channel mask to 3 channels to apply it to the BGR image
            mask_3channel = np.stack((binary_mask,) * 3, axis=-1)
            
            # Create a solid white background (BGR format)
            background_color = [255, 255, 255] 
            solid_background = np.full(img.shape, background_color, dtype=np.uint8)
            
            # Use the mask to place the person (img) on the white background
            processed_img = np.where(mask_3channel, img, solid_background)
        
        # --- Edge Detection with CLAHE Contrast Enhancement ---
        
        gray_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
        
        # 1. ADAPTIVE CONTRAST (CLAHE): Boost local features 
        # Increased clipLimit for stronger detail enhancement in the face area
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        high_contrast_img = clahe.apply(gray_img) 
        
        # 2. STRONG GAUSSIAN BLUR: Removes noise and texture (like fine hair details)
        # (13, 13) is a strong blur to eliminate insignificant stuff.
        blurred_img = cv2.GaussianBlur(high_contrast_img, (41, 41), 0)

        # 3. TIGHT CANNY THRESHOLDS: Only grab the strongest, essential lines
        # Using 0 as the lower threshold here can sometimes capture more detail, 
        # but combined with the strong blur, it should still result in clean lines.
        # We will use the original thresholds from the provided logic (100/200 on the blurred image).
        low_threshold = 100
        high_threshold = 200 
        edges = cv2.Canny(blurred_img, 0, 30)

        # 4. DILATION to Thicken and Connect Lines
        kernel = np.ones((5, 5), np.uint8) 
        dilated_edges = cv2.dilate(edges, kernel, iterations=2) 

        # Invert to get black lines on a white background
        line_drawing = cv2.bitwise_not(dilated_edges)

        # Save the result
        cv2.imwrite(output_path, line_drawing)
        print(f"Final minimal line drawing saved to: {output_path}")

image_to_minimal_line_drawing_final('divija.png', 'minimal_line_drawing_final.png')

Starting MediaPipe/CLAHE line drawing for: divija.png


I0000 00:00:1760842821.649733 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760842821.652119 48204329 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Final minimal line drawing saved to: minimal_line_drawing_final.png


In [37]:
def post_process_line_drawing(input_line_drawing_path, output_path="cleaned_line_drawing.png"):
    """
    Applies morphological operations and skeletonization to an existing line drawing
    to create a cleaner, more single-line type of drawing.
    """
    print(f"Starting post-processing for: {input_line_drawing_path}")

    # Load the line drawing (assuming it's black lines on a white background)
    img = cv2.imread(input_line_drawing_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Error: Could not read line drawing at {input_line_drawing_path}")
        return

    # Invert to make lines white on a black background for morphological operations
    # (Many CV operations work better on white foreground)
    inverted_img = cv2.bitwise_not(img)

    # Convert to binary (ensure values are strictly 0 or 255)
    _, binary_img = cv2.threshold(inverted_img, 127, 255, cv2.THRESH_BINARY)

    # 1. Closing: Dilation followed by Erosion
    # Helps close small gaps and merge nearby lines.
    # Kernel size will influence how much "merging" happens.
    close_kernel = np.ones((3, 3), np.uint8) # Adjust kernel size as needed
    closed_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, close_kernel, iterations=1)

    # 2. Opening: Erosion followed by Dilation
    # Helps remove small isolated noise pixels and thin out lines slightly.
    open_kernel = np.ones((3, 3), np.uint8) # Adjust kernel size as needed
    opened_img = cv2.morphologyEx(closed_img, cv2.MORPH_OPEN, open_kernel, iterations=1)
    
    # 3. Skeletonization (Medial Axis Transform)
    # This algorithm reduces binary objects (lines) to a single-pixel wide skeleton.
    # OpenCV doesn't have a direct `skeletonize` function, so we implement a common algorithm.
    skeleton = np.zeros(opened_img.shape, dtype=np.uint8)
    eroded = opened_img.copy()
    temp = np.zeros(opened_img.shape, dtype=np.uint8)
    
    # Define a cross-shaped structuring element for thinning
    kernel_thin = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))

    while True:
        eroded = cv2.erode(eroded, kernel_thin)
        temp = cv2.dilate(eroded, kernel_thin)
        temp = cv2.subtract(opened_img, temp)
        skeleton = cv2.bitwise_or(skeleton, temp)
        opened_img = eroded.copy()
        if cv2.countNonZero(opened_img) == 0:
            break
            
    # Optional: Remove very small contours (remaining noise/specks)
    # Find contours
    contours, _ = cv2.findContours(skeleton, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cleaned_skeleton = np.zeros_like(skeleton)
    min_area = 10 # Adjust this value: smaller values keep more detail, larger remove more noise

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area:
            cv2.drawContours(cleaned_skeleton, [contour], -1, 255, 1) # Draw white lines

    # Invert back to black lines on a white background
    final_drawing = cv2.bitwise_not(cleaned_skeleton)

    cv2.imwrite(output_path, final_drawing)
    print(f"Cleaned line drawing saved to: {output_path}")

post_process_line_drawing('minimal_line_drawing_final.png', 'cleaned_line_drawing.png')

Starting post-processing for: minimal_line_drawing_final.png
Cleaned line drawing saved to: cleaned_line_drawing.png


In [48]:
def image_to_cartoon_stylized_drawing(image_path, output_path="cartoon_stylized_output.png"):
    """
    Creates a very simple, stylized, almost cartoon-like drawing of a human 
    face and silhouette by treating both the facial features and the body 
    as contours (objects) to ensure smooth, continuous lines.
    """
    print(f"Starting Contour-Based Stylized Drawing for: {image_path}")

    # Initialize MediaPipe solutions
    mp_face_mesh = mp.solutions.face_mesh
    mp_selfie_segmentation = mp.solutions.selfie_segmentation
    
    # MODIFICATION: Separate the Outer Face Oval from Internal Features
    face_oval_set = [mp_face_mesh.FACEMESH_FACE_OVAL]
    
    # Internal features (Lips, Eyes, Nose, Eyebrows)
    internal_feature_sets = [
        mp_face_mesh.FACEMESH_LIPS,
        mp_face_mesh.FACEMESH_LEFT_EYE,
        mp_face_mesh.FACEMESH_RIGHT_EYE,
        mp_face_mesh.FACEMESH_NOSE,
        mp_face_mesh.FACEMESH_LEFT_EYEBROW,
        mp_face_mesh.FACEMESH_RIGHT_EYEBROW
    ]
    
    # BGR Black color for lines
    line_color = (0, 0, 0)
    line_thickness = 2 
    
    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    
    # Create a blank white canvas for the output drawing
    drawing = np.full((H, W, 3), 255, dtype=np.uint8) 
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 1. --- Contour-Based Drawing for Facial Features ---
    # We create a temporary mask for the features and then find contours
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results = face_mesh.process(img_rgb)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                
                # --- A. Draw Internal Features (Eyes, Lips, Nose) ---
                for feature_set in internal_feature_sets:
                    
                    # Create a fresh mask for each internal feature to ensure separation
                    feature_mask = np.zeros((H, W), dtype=np.uint8)
                    
                    points = []
                    for connection in feature_set:
                        p = face_landmarks.landmark[connection[0]]
                        points.append((int(p.x * W), int(p.y * H)))

                    # Use convex hull to define the shape and fill it solid white in the mask
                    points = np.array(points, np.int32)
                    
                    # We must check if we have enough points for convexHull and contour finding
                    if len(points) >= 3: 
                        hull = cv2.convexHull(points)
                        cv2.fillPoly(feature_mask, [hull], 255)

                        # Find and Draw Contours for this single internal feature
                        feature_contours, _ = cv2.findContours(feature_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        
                        for contour in feature_contours:
                            # Low epsilon for minimal simplification
                            epsilon = 0.0005 * cv2.arcLength(contour, True) 
                            approx_contour = cv2.approxPolyDP(contour, epsilon, True)
                            
                            cv2.drawContours(drawing, [approx_contour], 0, line_color, line_thickness)

                # --- B. Draw the Face Oval Contour Separately ---
                for feature_set in face_oval_set:
                    
                    # Create a mask for the oval only
                    oval_mask = np.zeros((H, W), dtype=np.uint8) 
                    
                    points = []
                    for connection in feature_set:
                        p = face_landmarks.landmark[connection[0]]
                        points.append((int(p.x * W), int(p.y * H)))

                    points = np.array(points, np.int32)
                    if len(points) >= 3:
                        hull = cv2.convexHull(points)
                        cv2.fillPoly(oval_mask, [hull], 255)

                        # Find and Draw Contours for the oval
                        oval_contours, _ = cv2.findContours(oval_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        
                        for contour in oval_contours:
                            epsilon = 0.0005 * cv2.arcLength(contour, True) 
                            approx_contour = cv2.approxPolyDP(contour, epsilon, True)
                            
                            # Draw the simplified contour boundary
                            cv2.drawContours(drawing, [approx_contour], 0, line_color, line_thickness)


    # 2. --- Draw Outer Silhouette (Hair, Head, Shirt) using Segmentation ---
    # This is the same logic as before, ensuring a clean outer line.
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as segmentor:
        
        segmentation_results = segmentor.process(img_rgb)
        
        if segmentation_results.segmentation_mask is not None:
            
            mask = segmentation_results.segmentation_mask
            mask_smooth = cv2.GaussianBlur(mask, (7, 7), 0)
            threshold = 0.5
            binary_mask = (mask_smooth * 255).astype(np.uint8)
            _, binary_mask = cv2.threshold(binary_mask, int(threshold * 255), 255, cv2.THRESH_BINARY)
            
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                
                # Epsilon for smooth silhouette (0.001 retains decent shape)
                epsilon = 0.001 * cv2.arcLength(largest_contour, True) 
                approx_contour = cv2.approxPolyDP(largest_contour, epsilon, True)
                
                # Draw the bold outer boundary
                cv2.drawContours(drawing, [approx_contour], 0, line_color, line_thickness + 2) 

    # Save the final result
    cv2.imwrite(output_path, drawing)
    print(f"Contour-based stylized drawing saved to: {output_path}")

image_to_cartoon_stylized_drawing('divija.png', 'cartoon_stylized_output.png')

Starting Contour-Based Stylized Drawing for: divija.png
Contour-based stylized drawing saved to: cartoon_stylized_output.png


I0000 00:00:1760841226.770654 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760841226.771394 48177902 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760841226.773054 48177901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760841226.796524 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760841226.797405 48177916 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [49]:
def image_to_hybrid_stylized_drawing(image_path, output_path="hybrid_stylized_output.png"):
    """
    Creates a hybrid stylized drawing by strictly limiting the CLAHE/Canny 
    edge detection ONLY to the facial region defined by the MediaPipe Face Mesh, 
    and combining it with a clean outer silhouette from Segmentation.
    """
    import numpy as np
    import cv2
    import mediapipe as mp
    import os

    print(f"Starting Hybrid Stylized Drawing for: {image_path}")

    # Initialize MediaPipe solutions
    mp_face_mesh = mp.solutions.face_mesh
    mp_selfie_segmentation = mp.solutions.selfie_segmentation
    
    # BGR Black color for lines
    line_color = (0, 0, 0)
    line_thickness = 2 
    
    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    
    # Create a blank white canvas for the output drawing
    drawing = np.full((H, W, 3), 255, dtype=np.uint8) 
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 0. --- Generate Face Mask (Region of Interest) ---
    face_mask_255 = np.zeros((H, W), dtype=np.uint8) # Mask starts black (0)
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results = face_mesh.process(img_rgb)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                
                # Get the points for the face oval contour
                points = []
                for connection in mp_face_mesh.FACEMESH_FACE_OVAL:
                    p = face_landmarks.landmark[connection[0]]
                    points.append((int(p.x * W), int(p.y * H)))
                
                # Use convex hull to define the face shape
                points = np.array(points, np.int32)
                if len(points) >= 3:
                    # Fill the inside of the face shape white (255)
                    hull = cv2.convexHull(points)
                    cv2.fillPoly(face_mask_255, [hull], 255)

    # 1. --- Edge Detection using CLAHE + Canny (Masked for Internal Detail) ---
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # A. ADAPTIVE CONTRAST (CLAHE): Aggressively boost local features
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8)) 
    high_contrast_img = clahe.apply(gray_img) 
    
    # B. LIGHT GAUSSIAN BLUR
    blurred_img = cv2.GaussianBlur(high_contrast_img, (5, 5), 0)

    # C. CANNY EDGE DETECTION: Find essential lines globally
    low_threshold = 20
    high_threshold = 80 
    internal_edges = cv2.Canny(blurred_img, low_threshold, high_threshold)

    # D. MASKING STEP: Apply the Face Mask to the Canny edges
    # Only keep the edges that are inside the white facial region (255)
    if np.any(face_mask_255):
        # Bitwise AND ensures edges are only kept where the mask is white (255)
        internal_edges_masked = cv2.bitwise_and(internal_edges, internal_edges, mask=face_mask_255)
    else:
        # If no face is detected, we fallback to just the global Canny edges (which might be noisy)
        internal_edges_masked = internal_edges

    # E. Draw the black Canny lines directly onto the white canvas
    line_mask_3ch = cv2.cvtColor(internal_edges_masked, cv2.COLOR_GRAY2BGR)
    
    # Wherever line_mask_3ch has a black line (0), the drawing canvas will be overwritten by 0 (black)
    drawing = np.minimum(drawing, line_mask_3ch)

    # 2. --- Draw Outer Silhouette (Hair, Head, Shirt) using Segmentation ---
    # This remains the same as before, ensuring a clean outer line.
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as segmentor:
        
        segmentation_results = segmentor.process(img_rgb)
        
        if segmentation_results.segmentation_mask is not None:
            
            mask = segmentation_results.segmentation_mask
            mask_smooth = cv2.GaussianBlur(mask, (7, 7), 0)
            threshold = 0.5
            binary_mask = (mask_smooth * 255).astype(np.uint8)
            _, binary_mask = cv2.threshold(binary_mask, int(threshold * 255), 255, cv2.THRESH_BINARY)
            
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                
                # Epsilon for smooth silhouette (0.001 retains decent shape)
                epsilon = 0.001 * cv2.arcLength(largest_contour, True) 
                approx_contour = cv2.approxPolyDP(largest_contour, epsilon, True)
                
                # Draw the bold outer boundary (thickness + 2)
                # Note: This is drawn *over* the Canny lines, ensuring the final silhouette is clean.
                cv2.drawContours(drawing, [approx_contour], 0, line_color, line_thickness + 2) 

    # Save the final result
    cv2.imwrite(output_path, drawing)
    print(f"Hybrid stylized drawing saved to: {output_path}")

image_to_hybrid_stylized_drawing('divija.png', 'hybrid_stylized_output.png')

Starting Hybrid Stylized Drawing for: divija.png
Hybrid stylized drawing saved to: hybrid_stylized_output.png


I0000 00:00:1760841372.010085 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760841372.010845 48180215 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760841372.012195 48180220 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760841372.052445 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760841372.053122 48180226 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [54]:
import mediapipe as mp
def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by selectively applying contrast enhancement 
    (CLAHE) and high-detail Canny edge detection only to the facial features (ROI defined 
    by Face Mesh), and then adding a minimal, low-detail outline for the hair and clothing 
    using the Selfie Segmentation mask.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mp_face_mesh = mp.solutions.face_mesh
    mp_selfie_segmentation = mp.solutions.selfie_segmentation

    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Feature ROI and CLAHE ---
    # The 'refine_landmarks=True' helps get better eye and lip contour details
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Calculate a tight bounding box around all face landmarks
            x_coords = [lm.x * W for lm in face_landmarks.landmark]
            y_coords = [lm.y * H for lm in face_landmarks.landmark]
            
            min_x, max_x = int(min(x_coords)), int(max(x_coords))
            min_y, max_y = int(min(y_coords)), int(max(y_coords))

            # Add padding and clamp to image boundaries to ensure smooth edges
            padding = int(W * 0.05)
            x1 = max(0, min_x - padding)
            y1 = max(0, min_y - padding)
            x2 = min(W, max_x + padding)
            y2 = min(H, max_y + padding)
            
            # Extract the ROI from the original image
            face_roi_area = img[y1:y2, x1:x2].copy()
            face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
            
            # --- Selective CLAHE and Canny for High-Detail Face ---
            
            # Apply ADAPTIVE CONTRAST (CLAHE): Increased clipLimit for stronger detail
            clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
            high_contrast_face = clahe.apply(face_roi_gray)
            
            # Apply Gaussian Blur (small kernel to retain facial detail, but remove fine noise)
            blurred_face = cv2.GaussianBlur(high_contrast_face, (5, 5), 0)
            
            # TIGHT CANNY THRESHOLDS for capturing essential lines like eyes, nose, mouth
            face_edges = cv2.Canny(blurred_face, 50, 150)
            
            # Dilate the face lines slightly to make them more visible
            kernel_face = np.ones((3, 3), np.uint8) 
            dilated_face_edges = cv2.dilate(face_edges, kernel_face, iterations=1) 
            
            # Invert the lines (black lines on white)
            inverted_face_edges = cv2.bitwise_not(dilated_face_edges)

            # Combine face edges onto the final canvas (black lines (0) overwrite white (255))
            current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
            final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_face_edges)

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Minimal Detail) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # Create a clean binary mask for the whole person (including hair and clothing)
            threshold = 0.5
            person_mask = (results_segmentation.segmentation_mask > threshold).astype(np.uint8)
            
            # Convert the mask to an image (white person shape on black background)
            person_outline_img = person_mask * 255 

            # Strong blur to simplify the outline (this only captures the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # Very broad Canny thresholds for minimal, loose outline lines
            low_threshold_outline = 10
            high_threshold_outline = 50 
            
            # Run Canny on the strongly blurred mask
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # Dilate to thicken the overall outline, making it the dominant silhouette
            kernel_outline = np.ones((7, 7), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=2) 
            
            # Invert the outline edges (black lines on white)
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the outline onto the final canvas using a bitwise AND.
            # This ensures that the low-detail outline complements the high-detail face,
            # and that lines from both are preserved.
            final_line_drawing = cv2.bitwise_and(final_line_drawing, inverted_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call (make sure you have an image named 'divija.png' in the same directory)
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')

Starting selective line drawing for: divija.png
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760841869.270815 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760841869.271533 48187720 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760841869.277457 48187719 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/divija/Divi Drive/workplace/Princeton/Sem 5/Carlab/carlab_2025/final_project/carlab/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
I0000 00:00:1760841869.290394 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 

In [55]:
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Indices for Eyes, Nose, and Mouth only (excluding the outer face contour/jawline)
# This list is used to define a tight bounding box around the key features.
FEATURE_INDICES = [
    # Nose (Nose bridge, tip)
    1, 6, 197,
    # Mouth (Upper lip, lower lip, corners)
    0, 17, 61, 291, 14, 308, 375,
    # Left Eye (Inner corner, outer corner, central points)
    33, 133, 163, 144, 246,
    # Right Eye (Inner corner, outer corner, central points)
    362, 263, 390, 373, 466
]

def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by selectively applying contrast enhancement 
    (CLAHE) and high-detail Canny edge detection only to the key facial features (ROI 
    defined by a subset of Face Mesh landmarks), and then adding a minimal, low-detail 
    outline for the hair and clothing using the Selfie Segmentation mask.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Feature ROI and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Calculate a tight bounding box *only* around the selected features
            x_coords = []
            y_coords = []
            
            for index in FEATURE_INDICES:
                lm = face_landmarks.landmark[index]
                x_coords.append(lm.x * W)
                y_coords.append(lm.y * H)
            
            min_x, max_x = int(min(x_coords)), int(max(x_coords))
            min_y, max_y = int(min(y_coords)), int(max(y_coords))

            # Add padding and clamp to image boundaries to ensure smooth edges
            padding = int(W * 0.05)
            x1 = max(0, min_x - padding)
            y1 = max(0, min_y - padding)
            x2 = min(W, max_x + padding)
            y2 = min(H, max_y + padding)
            
            # Extract the ROI from the original image
            face_roi_area = img[y1:y2, x1:x2].copy()
            face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
            
            # --- Selective CLAHE and Canny for High-Detail Features ---
            
            # Apply ADAPTIVE CONTRAST (CLAHE): Increased clipLimit for stronger detail
            clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
            high_contrast_face = clahe.apply(face_roi_gray)
            
            # Apply Gaussian Blur (small kernel to retain facial detail, but remove fine noise)
            blurred_face = cv2.GaussianBlur(high_contrast_face, (5, 5), 0)
            
            # TIGHT CANNY THRESHOLDS for capturing essential lines like eyes, nose, mouth
            face_edges = cv2.Canny(blurred_face, 50, 150)
            
            # Dilate the face lines slightly to make them more visible
            kernel_face = np.ones((3, 3), np.uint8) 
            dilated_face_edges = cv2.dilate(face_edges, kernel_face, iterations=1) 
            
            # Invert the lines (black lines on white)
            inverted_face_edges = cv2.bitwise_not(dilated_face_edges)

            # Combine face edges onto the final canvas (black lines (0) overwrite white (255))
            current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
            final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_face_edges)

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Minimal Detail) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # Create a clean binary mask for the whole person (including hair and clothing)
            threshold = 0.5
            person_mask = (results_segmentation.segmentation_mask > threshold).astype(np.uint8)
            
            # Convert the mask to an image (white person shape on black background)
            person_outline_img = person_mask * 255 

            # Strong blur to simplify the outline (this only captures the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # Very broad Canny thresholds for minimal, loose outline lines
            low_threshold_outline = 10
            high_threshold_outline = 50 
            
            # Run Canny on the strongly blurred mask
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # Dilate to thicken the overall outline, making it the dominant silhouette
            kernel_outline = np.ones((7, 7), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=2) 
            
            # Invert the outline edges (black lines on white)
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the outline onto the final canvas using a bitwise AND.
            # This ensures that the low-detail outline complements the high-detail face,
            # and that lines from both are preserved.
            final_line_drawing = cv2.bitwise_and(final_line_drawing, inverted_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')

Starting selective line drawing for: divija.png
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760842077.552652 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760842077.553956 48191226 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760842077.557301 48191229 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/divija/Divi Drive/workplace/Princeton/Sem 5/Carlab/carlab_2025/final_project/carlab/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
I0000 00:00:1760842077.567557 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 

In [56]:

# Initialize MediaPipe solutions (all required imports in one place)
# We need Face Mesh for precise feature ROI, and Selfie Segmentation for the body/hair outline.
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Indices for Eyes, Nose, and Mouth only (excluding the outer face contour/jawline)
# This list is used to define a tight bounding box around the key features.
FEATURE_INDICES = [
    # Nose (Nose bridge, tip)
    1, 6, 197,
    # Mouth (Upper lip, lower lip, corners)
    0, 17, 61, 291, 14, 308, 375,
    # Left Eye (Inner corner, outer corner, central points)
    33, 133, 163, 144, 246,
    # Right Eye (Inner corner, outer corner, central points)
    362, 263, 390, 373, 466
]

def image_to_minimal_line_drawing_enhanced(image_path, output_path, debug_mode=False):
    """
    Creates a highly stylized line drawing by selectively applying contrast enhancement 
    (CLAHE) and high-detail Canny edge detection only to the key facial features (ROI 
    defined by a subset of Face Mesh landmarks), and then adding a minimal, low-detail 
    outline for the hair and clothing using the Selfie Segmentation mask.
    
    If debug_mode is True, it saves the cropped ROI image and stops execution.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Feature ROI and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Calculate a tight bounding box *only* around the selected features
            x_coords = []
            y_coords = []
            
            for index in FEATURE_INDICES:
                lm = face_landmarks.landmark[index]
                x_coords.append(lm.x * W)
                y_coords.append(lm.y * H)
            
            min_x, max_x = int(min(x_coords)), int(max(x_coords))
            min_y, max_y = int(min(y_coords)), int(max(y_coords))

            # Add padding and clamp to image boundaries to ensure smooth edges
            padding = int(W * 0.05)
            x1 = max(0, min_x - padding)
            y1 = max(0, min_y - padding)
            x2 = min(W, max_x + padding)
            y2 = min(H, max_y + padding)
            
            # Extract the ROI from the original image
            face_roi_area = img[y1:y2, x1:x2].copy()
            
            # --- DEBUG MODE: Save ROI and stop ---
            if debug_mode:
                debug_path = output_path.replace('.png', '_roi_only.png')
                cv2.imwrite(debug_path, face_roi_area)
                print(f"DEBUG MODE: ROI extracted to: {debug_path}")
                print(f"DEBUG MODE: ROI coordinates (x1, y1, x2, y2): ({x1}, {y1}, {x2}, {y2})")
                return # Stop further processing

            # --- Selective CLAHE and Canny for High-Detail Features ---
            face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
            
            # Apply ADAPTIVE CONTRAST (CLAHE): Increased clipLimit for stronger detail
            clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
            high_contrast_face = clahe.apply(face_roi_gray)
            
            # Apply Gaussian Blur (small kernel to retain facial detail, but remove fine noise)
            blurred_face = cv2.GaussianBlur(high_contrast_face, (5, 5), 0)
            
            # TIGHT CANNY THRESHOLDS for capturing essential lines like eyes, nose, mouth
            face_edges = cv2.Canny(blurred_face, 50, 150)
            
            # Dilate the face lines slightly to make them more visible
            kernel_face = np.ones((3, 3), np.uint8) 
            dilated_face_edges = cv2.dilate(face_edges, kernel_face, iterations=1) 
            
            # Invert the lines (black lines on white)
            inverted_face_edges = cv2.bitwise_not(dilated_face_edges)

            # Combine face edges onto the final canvas (black lines (0) overwrite white (255))
            current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
            final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_face_edges)

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Minimal Detail) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # Create a clean binary mask for the whole person (including hair and clothing)
            threshold = 0.5
            person_mask = (results_segmentation.segmentation_mask > threshold).astype(np.uint8)
            
            # Convert the mask to an image (white person shape on black background)
            person_outline_img = person_mask * 255 

            # Strong blur to simplify the outline (this only captures the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # Very broad Canny thresholds for minimal, loose outline lines
            low_threshold_outline = 10
            high_threshold_outline = 50 
            
            # Run Canny on the strongly blurred mask
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # Dilate to thicken the overall outline, making it the dominant silhouette
            kernel_outline = np.ones((7, 7), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=2) 
            
            # Invert the outline edges (black lines on white)
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the outline onto the final canvas using a bitwise AND.
            # This ensures that the low-detail outline complements the high-detail face,
            # and that lines from both are preserved.
            final_line_drawing = cv2.bitwise_and(final_line_drawing, inverted_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
# image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')

# Example call for DEBUG MODE (Uncomment this line to test the ROI)
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png', debug_mode=True)

Starting selective line drawing for: divija.png
DEBUG MODE: ROI extracted to: selective_line_drawing_roi_only.png
DEBUG MODE: ROI coordinates (x1, y1, x2, y2): (1320, 1246, 2312, 2138)


I0000 00:00:1760842167.834645 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760842167.835684 48192795 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760842167.838968 48192797 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [85]:
# Initialize MediaPipe solutions (all required imports in one place)
# We need Face Mesh for precise feature ROI, and Selfie Segmentation for the body/hair outline.
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Feature groups for distinct processing (Eyes, Nose, Mouth).
# Each group will be processed with its own tight bounding box.
FEATURE_GROUPS = {
    # Indices covering both eyes, excluding the face contour
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463], 
    
    # Indices covering the nose bridge, tip, and base
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420], 
    
    # Indices covering the lips and corners of the mouth
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415] ,
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
    
    
}

def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by applying CLAHE and Canny edge detection 
    independently to multiple small facial feature ROIs, then adding a minimal 
    outline for the hair and clothing.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Loop over each feature group to process them individually
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords = []
                y_coords = []
                
                # Calculate a tight bounding box *only* around the selected feature group
                for index in indices:
                    lm = face_landmarks.landmark[index]
                    x_coords.append(lm.x * W)
                    y_coords.append(lm.y * H)
                
                if not x_coords: continue # Skip if no landmarks are found
                
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                # Use a small fixed padding for these tiny ROIs
                padding = 15 
                x1 = max(0, min_x - padding)
                y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding)
                y2 = min(H, max_y + padding)
                
                # Ensure ROI is valid
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the original image
                face_roi_area = img[y1:y2, x1:x2].copy()
                face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
                
                # --- Selective CLAHE and Canny for High-Detail Features ---
                
                # Apply ADAPTIVE CONTRAST (CLAHE)
                clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                high_contrast_feature = clahe.apply(face_roi_gray)
                
                # Apply Gaussian Blur (kernel size adjusted slightly per feature type if needed, or stick to small (5, 5))
                blurred_feature = cv2.GaussianBlur(high_contrast_feature, (11, 11), 0)
                
                # TIGHT CANNY THRESHOLDS
                feature_edges = cv2.Canny(blurred_feature, 0, 100)
                
                # Dilate the feature lines slightly
                kernel_feature = np.ones((3, 3), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                # Invert the lines (black lines on white)
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)

                # Combine feature edges onto the final canvas (black lines (0) overwrite white (255))
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Minimal Detail) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # Create a clean binary mask for the whole person (including hair and clothing)
            threshold = 0.5
            person_mask = (results_segmentation.segmentation_mask > threshold).astype(np.uint8)
            
            # Convert the mask to an image (white person shape on black background)
            person_outline_img = person_mask * 255 

            # Strong blur to simplify the outline (this only captures the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # Very broad Canny thresholds for minimal, loose outline lines
            low_threshold_outline = 10
            high_threshold_outline = 50 
            
            # Run Canny on the strongly blurred mask
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # Dilate to thicken the overall outline, making it the dominant silhouette
            kernel_outline = np.ones((7, 7), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=2) 
            
            # Invert the outline edges (black lines on white)
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the outline onto the final canvas using a bitwise AND.
            # This ensures that the low-detail outline complements the high-detail face,
            # and that lines from both are preserved.
            final_line_drawing = cv2.bitwise_and(final_line_drawing, inverted_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')


Starting selective line drawing for: divija.png
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760843085.202883 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760843085.203591 48208784 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760843085.207236 48208783 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760843085.215987 48084326 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760843085.216621 48208796 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [41]:
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Feature groups for distinct processing (Eyes, Nose, Mouth).
# Each group will be processed with its own tight bounding box.
FEATURE_GROUPS = {
    # Indices covering both eyes, excluding the face contour
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463], 
    
    # INDICES UPDATED: Added points for better coverage of nostrils and the nose base.
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420, 129, 358, 64, 294, 98, 327, 118, 347], 
    
    # Indices covering the lips and corners of the mouth
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415],
    
    # UPDATED GROUP: Simplified indices to reliably anchor the region between nose (2) and lips (13, 14).
    # This creates a vertical bounding box for the philtrum.
    "philtrum": [2, 13, 14, 172, 406],
    
    # Indices covering both left and right eyebrows
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
    

}

# Define Canny thresholds based on feature importance/noise level
# Lower thresholds (e.g., 50, 150) capture more detail (high sensitivity)
# Higher thresholds (e.g., 80, 200) capture only strong, clean edges (low sensitivity/less noise)
CANNY_THRESHOLDS = {
    "eyes": (50, 120),       # High detail for eyes
    "nose": (50, 120),       # High detail for nose
    "mouth": (50, 120),      # High detail for lips
    "philtrum": (60, 120),   # Moderate detail for philtrum/creases
    "eyebrows": (0, 40),   # Low detail/Less noise for eyebrows
}
GAUSSIAN_BLUR_SIZES = {
    "eyes": (13, 13),
    "nose": (11, 11),
    "mouth": (11, 11),
    "philtrum": (31, 31),
    "eyebrows": (31, 31),
}

def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by applying CLAHE and Canny edge detection 
    independently to multiple small facial feature ROIs, then adding a minimal 
    outline for the hair and clothing.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Loop over each feature group to process them individually
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords = []
                y_coords = []
                
                # Calculate a tight bounding box *only* around the selected feature group
                for index in indices:
                    lm = face_landmarks.landmark[index]
                    x_coords.append(lm.x * W)
                    y_coords.append(lm.y * H)
                
                if not x_coords: continue # Skip if no landmarks are found
                
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                # Use a small fixed padding for these tiny ROIs
                padding = 15
                if feature_name == "nose" or feature_name == "philtrum":
                    # Increase padding for the nose and philtrum to prevent harsh cutoffs and ensure better blending
                    padding = 25 

                x1 = max(0, min_x - padding)
                y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding)
                y2 = min(H, max_y + padding)
                
                # Ensure ROI is valid
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the original image
                face_roi_area = img[y1:y2, x1:x2].copy()
                
                # --- DEBUG: Save the cropped ROI image for inspection ---
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_roi.png')
                cv2.imwrite(roi_debug_path, face_roi_area)
                print(f"DEBUG: Saved ROI for {feature_name} to: {roi_debug_path}")
                # --- END DEBUG ---

                face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
                
                # --- Selective CLAHE and Canny for High-Detail Features ---
                
                # Apply ADAPTIVE CONTRAST (CLAHE)
                clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                high_contrast_feature = clahe.apply(face_roi_gray)
                
                # Apply Gaussian Blur (kernel size adjusted slightly per feature type if needed, or stick to small (5, 5))
                blurred_feature = cv2.GaussianBlur(high_contrast_feature, GAUSSIAN_BLUR_SIZES.get(feature_name, (5, 5)), 0)
                
                # --- APPLY DYNAMIC CANNY THRESHOLDS ---
                low_thresh, high_thresh = CANNY_THRESHOLDS.get(feature_name, (50, 150))
                feature_edges = cv2.Canny(blurred_feature, low_thresh, high_thresh)
                # --- END DYNAMIC CANNY THRESHOLDS ---
                
                # Dilate the feature lines slightly
                kernel_feature = np.ones((7, 7), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                # Invert the lines (black lines on white)
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)

                # Combine feature edges onto the final canvas (black lines (0) overwrite white (255))
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)

     # --- 2. Selfie Segmentation for Hair and Clothing Outline (Minimal Detail) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # Create a clean binary mask for the whole person (including hair and clothing)
            threshold = 0.5
            person_mask = (results_segmentation.segmentation_mask > threshold).astype(np.uint8)
            
            # Convert the mask to an image (white person shape on black background)
            person_outline_img = person_mask * 255 

            # Strong blur to simplify the outline (this only captures the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # Very broad Canny thresholds for minimal, loose outline lines
            low_threshold_outline = 10
            high_threshold_outline = 50
         
            
            # Run Canny on the strongly blurred mask
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # Dilate to thicken the overall outline, making it the dominant silhouette
            kernel_outline = np.ones((7, 7), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=2) 
            
            # Invert the outline edges (black lines on white)
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the outline onto the final canvas using a bitwise AND.
            # This ensures that the low-detail outline complements the high-detail face,
            # and that lines from both are preserved.
            final_line_drawing = cv2.bitwise_and(final_line_drawing, inverted_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')


Starting selective line drawing for: divija.png
DEBUG: Saved ROI for eyes to: selective_line_drawing_eyes_roi.png
DEBUG: Saved ROI for nose to: selective_line_drawing_nose_roi.png
DEBUG: Saved ROI for mouth to: selective_line_drawing_mouth_roi.png
DEBUG: Saved ROI for philtrum to: selective_line_drawing_philtrum_roi.png
DEBUG: Saved ROI for eyebrows to: selective_line_drawing_eyebrows_roi.png
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760845883.300894 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760845883.301598 48257597 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760845883.305178 48257596 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760845883.330986 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760845883.331650 48257610 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [39]:

# Initialize MediaPipe solutions (all required imports in one place)
# We need Face Mesh for precise feature ROI, and Selfie Segmentation for the body/hair outline.
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Feature groups for distinct processing (Eyes, Nose, Mouth).
# Each group will be processed with its own tight bounding box.
FEATURE_GROUPS = {
    # Indices covering both eyes, excluding the face contour
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463], 
    
    # INDICES UPDATED: Added points for better coverage of nostrils and the nose base.
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420, 129, 358, 64, 294, 98, 327, 118, 347], 
    
    # Indices covering the lips and corners of the mouth
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415],
    
    # UPDATED GROUP: Simplified indices to reliably anchor the region between nose (2) and lips (13, 14).
    # This creates a vertical bounding box for the philtrum.
    "philtrum": [2, 13, 14, 172, 406],
    
    # Indices covering both left and right eyebrows
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
    
    # NOTE: The 'ears' group has been removed per your request.
}

# Define Canny thresholds based on feature importance/noise level
# Lower thresholds (e.g., 50, 120) capture more detail (high sensitivity)
# Higher thresholds (e.g., 80, 200) capture only strong, clean edges (low sensitivity/less noise)
CANNY_THRESHOLDS = {
    "eyes": (50, 120),       # High detail for eyes
    "nose": (50, 120),       # High detail for nose
    "mouth": (50, 120),      # High detail for lips
    "philtrum": (60, 120),   # Moderate detail for philtrum/creases
    "eyebrows": (0, 40),     # Aggressive/Low-end settings to detect smooth outlines after heavy blur
}

# Define Gaussian Blur kernel sizes (must be odd numbers) based on required detail
# Larger kernels (e.g., 31) remove more texture/noise, retaining only gross shape.
# Smaller kernels (e.g., 11) retain finer detail.
GAUSSIAN_BLUR_SIZES = {
    "eyes": (13, 13),
    "nose": (11, 11),
    "mouth": (11, 11),
    "philtrum": (31, 31),
    "eyebrows": (31, 31),
}


def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by applying CLAHE and Canny edge detection 
    independently to multiple small facial feature ROIs, then adding a minimal 
    outline for the hair and clothing.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Loop over each feature group to process them individually
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords = []
                y_coords = []
                
                # Calculate a tight bounding box *only* around the selected feature group
                for index in indices:
                    lm = face_landmarks.landmark[index]
                    x_coords.append(lm.x * W)
                    y_coords.append(lm.y * H)
                
                if not x_coords: continue # Skip if no landmarks are found
                
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                # Use a small fixed padding for these tiny ROIs
                padding = 15
                if feature_name == "nose" or feature_name == "philtrum":
                    # Increase padding for the nose and philtrum to prevent harsh cutoffs and ensure better blending
                    padding = 25 

                x1 = max(0, min_x - padding)
                y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding)
                y2 = min(H, max_y + padding)
                
                # Ensure ROI is valid
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the original image
                face_roi_area = img[y1:y2, x1:x2].copy()
                
                # --- DEBUG: Save the cropped ROI image for inspection ---
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_roi.png')
                cv2.imwrite(roi_debug_path, face_roi_area)
                print(f"DEBUG: Saved ROI for {feature_name} to: {roi_debug_path}")
                # --- END DEBUG ---

                face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
                
                # --- Selective CLAHE and Canny for High-Detail Features ---
                
                # Apply ADAPTIVE CONTRAST (CLAHE)
                clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                high_contrast_feature = clahe.apply(face_roi_gray)
                
                # Apply Dynamic Gaussian Blur
                blur_size = GAUSSIAN_BLUR_SIZES.get(feature_name, (5, 5))
                blurred_feature = cv2.GaussianBlur(high_contrast_feature, blur_size, 0)
                
                # --- APPLY DYNAMIC CANNY THRESHOLDS ---
                low_thresh, high_thresh = CANNY_THRESHOLDS.get(feature_name, (50, 150))
                feature_edges = cv2.Canny(blurred_feature, low_thresh, high_thresh)
                # --- END DYNAMIC CANNY THRESHOLDS ---
                
                # Dilate the feature lines slightly
                kernel_feature = np.ones((3, 3), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                # Invert the lines (black lines on white)
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)

                # Combine feature edges onto the final canvas (black lines (0) overwrite white (255))
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)

    # --- 2. Selfie Segmentation for Hair, Clothing, and Hair/Face Boundary Outline (Inner Edges) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # Create a clean version of the person on a white background (BGR)
            threshold = 0.5
            person_mask = results_segmentation.segmentation_mask > threshold
            mask_3channel = np.stack((person_mask,) * 3, axis=-1)
            
            background_color = [255, 255, 255] 
            solid_background = np.full(img.shape, background_color, dtype=np.uint8)
            # Isolate the original image of the person against a white background
            person_on_white = np.where(mask_3channel, img, solid_background)
            
            # --- COLOR-BASED EDGE DETECTION (HSV Saturation Channel) ---
            # Convert to HSV to better isolate hair/skin color contrast
            person_hsv = cv2.cvtColor(person_on_white, cv2.COLOR_BGR2HSV)
            
            # Use the Saturation channel (index 1), which typically has low values for skin 
            # and higher values for hair/clothing/background, providing a strong boundary contrast.
            person_saturation = person_hsv[:, :, 1]

            # Apply a lighter blur (9, 9) to smooth noise while retaining the hair boundary edge.
            blurred_person = cv2.GaussianBlur(person_saturation, (9, 9), 0)

            # Use Canny on the Saturation channel with thresholds tuned for color contrast
            low_threshold_outline = 40 
            high_threshold_outline = 120 
            
            outline_edges = cv2.Canny(blurred_person, low_threshold_outline, high_threshold_outline)
            
            # --- CONTOUR FILTERING HEURISTIC (NEW) ---
            # Invert the Canny output temporarily for contour finding (contours look for white on black)
            # We don't need a full inversion here since Canny output is effectively a binary image
            
            # Find contours (lines)
            contours, _ = cv2.findContours(outline_edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

            # Create a blank white canvas for the filtered outline
            filtered_outline = np.ones((H, W), dtype=np.uint8) * 255
            
            # Heuristic: Filter by area to keep only the prominent, continuous lines 
            # (main silhouette, hair edge, clothing folds) and discard small noise.
            min_outline_area = 100 # Adjust this threshold based on image resolution if needed

            for contour in contours:
                if cv2.contourArea(contour) > min_outline_area:
                    # Draw the long continuous lines onto the filtered canvas (in black, thickness 1 for thin lines)
                    cv2.drawContours(filtered_outline, [contour], -1, 0, 1) 
            
            # The filtered_outline now contains the continuous hair/clothing lines (black on white)
            
            # Combine the outline onto the final canvas, preserving all lines.
            final_line_drawing = np.minimum(final_line_drawing, filtered_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')


Starting selective line drawing for: divija.png
DEBUG: Saved ROI for eyes to: selective_line_drawing_eyes_roi.png
DEBUG: Saved ROI for nose to: selective_line_drawing_nose_roi.png
DEBUG: Saved ROI for mouth to: selective_line_drawing_mouth_roi.png
DEBUG: Saved ROI for philtrum to: selective_line_drawing_philtrum_roi.png
DEBUG: Saved ROI for eyebrows to: selective_line_drawing_eyebrows_roi.png
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760845705.839927 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760845705.840647 48254751 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760845705.844306 48254753 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760845705.867318 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760845705.868016 48254771 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [44]:
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Feature groups for distinct processing (Eyes, Nose, Mouth).
# Each group will be processed with its own tight bounding box.
FEATURE_GROUPS = {
    # Indices covering both eyes, excluding the face contour
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463], 
    
    # INDICES UPDATED: Added points for better coverage of nostrils and the nose base.
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420, 129, 358, 64, 294, 98, 327, 118, 347], 
    
    # Indices covering the lips and corners of the mouth
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415],
    
    # UPDATED GROUP: Simplified indices to reliably anchor the region between nose (2) and lips (13, 14).
    # This creates a vertical bounding box for the philtrum.
    "philtrum": [2, 13, 14, 172, 406],
    
    # Indices covering both left and right eyebrows
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
    

}

# Define Canny thresholds based on feature importance/noise level
# Lower thresholds (e.g., 50, 150) capture more detail (high sensitivity)
# Higher thresholds (e.g., 80, 200) capture only strong, clean edges (low sensitivity/less noise)
CANNY_THRESHOLDS = {
    "eyes": (50, 120),       # High detail for eyes
    "nose": (50, 120),       # High detail for nose
    "mouth": (50, 120),      # High detail for lips
    "philtrum": (60, 120),   # Moderate detail for philtrum/creases
    "eyebrows": (0, 40),   # Low detail/Less noise for eyebrows
}
GAUSSIAN_BLUR_SIZES = {
    "eyes": (13, 13),
    "nose": (11, 11),
    "mouth": (11, 11),
    "philtrum": (31, 31),
    "eyebrows": (61, 61),
}

def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by applying CLAHE and Canny edge detection 
    independently to multiple small facial feature ROIs, then adding a minimal 
    outline for the hair and clothing.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Loop over each feature group to process them individually
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords = []
                y_coords = []
                
                # Calculate a tight bounding box *only* around the selected feature group
                for index in indices:
                    lm = face_landmarks.landmark[index]
                    x_coords.append(lm.x * W)
                    y_coords.append(lm.y * H)
                
                if not x_coords: continue # Skip if no landmarks are found
                
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                # Use a small fixed padding for these tiny ROIs
                padding = 15
                if feature_name == "nose" or feature_name == "philtrum":
                    # Increase padding for the nose and philtrum to prevent harsh cutoffs and ensure better blending
                    padding = 25 

                x1 = max(0, min_x - padding)
                y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding)
                y2 = min(H, max_y + padding)
                
                # Ensure ROI is valid
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the original image
                face_roi_area = img[y1:y2, x1:x2].copy()
                
                # --- DEBUG: Save the cropped ROI image for inspection ---
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_roi.png')
                cv2.imwrite(roi_debug_path, face_roi_area)
                print(f"DEBUG: Saved ROI for {feature_name} to: {roi_debug_path}")
                # --- END DEBUG ---

                face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
                
                # --- Selective CLAHE and Canny for High-Detail Features ---
                
                # Apply ADAPTIVE CONTRAST (CLAHE)
                clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                high_contrast_feature = clahe.apply(face_roi_gray)
                
                # Apply Gaussian Blur (kernel size adjusted slightly per feature type if needed, or stick to small (5, 5))
                blurred_feature = cv2.GaussianBlur(high_contrast_feature, GAUSSIAN_BLUR_SIZES.get(feature_name, (5, 5)), 0)
                
                # --- APPLY DYNAMIC CANNY THRESHOLDS ---
                low_thresh, high_thresh = CANNY_THRESHOLDS.get(feature_name, (50, 150))
                feature_edges = cv2.Canny(blurred_feature, low_thresh, high_thresh)
                # --- END DYNAMIC CANNY THRESHOLDS ---
                
                # Dilate the feature lines slightly
                kernel_feature = np.ones((7, 7), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                # Invert the lines (black lines on white)
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)

                # Combine feature edges onto the final canvas (black lines (0) overwrite white (255))
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)

 # --- 2. Selfie Segmentation for Hair and Clothing Outline ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # Create a clean version of the person on a white background (BGR)
            threshold = 0.5
            person_mask = results_segmentation.segmentation_mask > threshold
            mask_3channel = np.stack((person_mask,) * 3, axis=-1)
            
            background_color = [255, 255, 255] 
            solid_background = np.full(img.shape, background_color, dtype=np.uint8)
            # Isolate the original image of the person against a white background
            person_on_white = np.where(mask_3channel, img, solid_background)
            
            # ----------------------------------------------------------------------
            # 2A. OUTERMOST SILHOUETTE (Simple Shape Detection - Grey Channel Logic)
            # ----------------------------------------------------------------------
            
            # Use the segmentation mask image (white on black)
            person_outline_img = person_mask.astype(np.uint8) * 255 

            # Strong blur to simplify the outline (this only captures the gross shape)
            blurred_outline_simple = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # Very broad Canny thresholds for minimal, loose outline lines
            low_threshold_outline_simple = 10
            high_threshold_outline_simple = 50 
            
            # Run Canny on the strongly blurred mask
            outline_edges_simple = cv2.Canny(blurred_outline_simple, low_threshold_outline_simple, high_threshold_outline_simple)
            
            # Light dilation to ensure the line is continuous
            kernel_outline_simple = np.ones((5, 5), np.uint8) 
            dilated_outline_edges_simple = cv2.dilate(outline_edges_simple, kernel_outline_simple, iterations=1) 
            
            # Invert the outline edges (black lines on white)
            inverted_outline_simple = cv2.bitwise_not(dilated_outline_edges_simple)
            
            # Combine the simple outline onto the final canvas
            final_line_drawing = np.minimum(final_line_drawing, inverted_outline_simple)
            
            # ----------------------------------------------------------------------
            # 2B. INNER DETAILS (Hair/Face Boundary & Clothing Folds - HSV Saturation/Contour Logic)
            # ----------------------------------------------------------------------
            
            # Convert to HSV to better isolate hair/skin color contrast
            person_hsv = cv2.cvtColor(person_on_white, cv2.COLOR_BGR2HSV)
            
            # Use the Saturation channel (index 1) for strong color boundaries
            person_saturation = person_hsv[:, :, 1]

            # Apply a lighter blur to smooth noise while retaining the hair boundary edge.
            blurred_person_inner = cv2.GaussianBlur(person_saturation, (9, 9), 0)

            # Use Canny on the Saturation channel with thresholds tuned for color contrast
            low_threshold_inner = 40 
            high_threshold_inner = 120 
            
            outline_edges_inner = cv2.Canny(blurred_person_inner, low_threshold_inner, high_threshold_inner)
            
            # --- CONTOUR FILTERING HEURISTIC ---
            # Find contours (lines)
            contours_inner, _ = cv2.findContours(outline_edges_inner, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

            # Create a blank white canvas for the filtered outline
            filtered_outline_inner = np.ones((H, W), dtype=np.uint8) * 255
            
            # Heuristic: Filter by area to keep only the prominent, continuous lines 
            min_outline_area = 100 

            for contour in contours_inner:
                if cv2.contourArea(contour) > min_outline_area:
                    # Draw the long continuous lines onto the filtered canvas (in black, thickness 1 for thin lines)
                    cv2.drawContours(filtered_outline_inner, [contour], -1, 0, 1) 
            
            # Combine the inner details onto the final canvas, preserving all lines.
            final_line_drawing = np.minimum(final_line_drawing, filtered_outline_inner)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')


Starting selective line drawing for: divija.png
DEBUG: Saved ROI for eyes to: selective_line_drawing_eyes_roi.png
DEBUG: Saved ROI for nose to: selective_line_drawing_nose_roi.png
DEBUG: Saved ROI for mouth to: selective_line_drawing_mouth_roi.png
DEBUG: Saved ROI for philtrum to: selective_line_drawing_philtrum_roi.png
DEBUG: Saved ROI for eyebrows to: selective_line_drawing_eyebrows_roi.png
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760846697.619743 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760846697.621959 48271852 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760846697.625715 48271854 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760846697.652031 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760846697.653263 48271866 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [64]:
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Feature groups for distinct processing (Eyes, Nose, Mouth).
# Each group will be processed with its own tight bounding box.
FEATURE_GROUPS = {
    # Indices covering both eyes, excluding the face contour
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463], 
    
    # INDICES UPDATED: Added points for better coverage of nostrils and the nose base.
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420, 129, 358, 64, 294, 98, 327, 118, 347], 
    
    # Indices covering the lips and corners of the mouth
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415],
    
    # UPDATED GROUP: Simplified indices to reliably anchor the region between nose (2) and lips (13, 14).
    # This creates a vertical bounding box for the philtrum.
    "philtrum": [2, 13, 14, 172, 406, ],
    
    # Indices covering both left and right eyebrows
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
    

}

# Define Canny thresholds based on feature importance/noise level
# Lower thresholds (e.g., 50, 150) capture more detail (high sensitivity)
# Higher thresholds (e.g., 80, 200) capture only strong, clean edges (low sensitivity/less noise)
CANNY_THRESHOLDS = {
    "eyes": (50, 120),       # High detail for eyes
    "nose": (50, 120),       # High detail for nose
    "mouth": (50, 120),      # High detail for lips
    "philtrum": (60, 120),   # Moderate detail for philtrum/creases
    "eyebrows": (10, 30),   # Low detail/Less noise for eyebrows
}
GAUSSIAN_BLUR_SIZES = {
    "eyes": (13, 13),
    "nose": (11, 11),
    "mouth": (11, 11),
    "philtrum": (31, 31),
    "eyebrows": (31, 31),
}

def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by applying CLAHE and Canny edge detection 
    independently to multiple small facial feature ROIs, then adding a minimal 
    outline for the hair and clothing.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Loop over each feature group to process them individually
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords = []
                y_coords = []
                
                # Calculate a tight bounding box *only* around the selected feature group
                for index in indices:
                    lm = face_landmarks.landmark[index]
                    x_coords.append(lm.x * W)
                    y_coords.append(lm.y * H)
                
                if not x_coords: continue # Skip if no landmarks are found
                
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                # Use a small fixed padding for these tiny ROIs
                padding = 15
                if feature_name == "nose" or feature_name == "philtrum":
                    # Increase padding for the nose and philtrum to prevent harsh cutoffs and ensure better blending
                    padding = 25 

                x1 = max(0, min_x - padding)
                y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding)
                y2 = min(H, max_y + padding)
                
                # Ensure ROI is valid
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the original image
                face_roi_area = img[y1:y2, x1:x2].copy()
                
                # --- DEBUG: Save the cropped ROI image for inspection ---
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_roi.png')
                cv2.imwrite(roi_debug_path, face_roi_area)
                print(f"DEBUG: Saved ROI for {feature_name} to: {roi_debug_path}")
                # --- END DEBUG ---

                face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
                
                # --- Selective CLAHE and Canny for High-Detail Features ---
                
                # Apply ADAPTIVE CONTRAST (CLAHE)
                clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                high_contrast_feature = clahe.apply(face_roi_gray)
                
                # Apply Gaussian Blur (kernel size adjusted slightly per feature type if needed, or stick to small (5, 5))
                blurred_feature = cv2.GaussianBlur(high_contrast_feature, GAUSSIAN_BLUR_SIZES.get(feature_name, (5, 5)), 0)
                
                # --- APPLY DYNAMIC CANNY THRESHOLDS ---
                low_thresh, high_thresh = CANNY_THRESHOLDS.get(feature_name, (50, 150))
                feature_edges = cv2.Canny(blurred_feature, low_thresh, high_thresh)
                # --- END DYNAMIC CANNY THRESHOLDS ---
                
                # Dilate the feature lines slightly
                kernel_feature = np.ones((7, 7), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                # Invert the lines (black lines on white)
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)

                # Combine feature edges onto the final canvas (black lines (0) overwrite white (255))
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Simplified Mask Approach) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # --- SIMPLIFIED OUTER OUTLINE (Based on Mask Only) ---
            threshold = 0.5
            person_mask = results_segmentation.segmentation_mask > threshold
            
            # 1. Isolate the mask image (white person shape on black background)
            person_outline_img = person_mask.astype(np.uint8) * 255 

            # 2. Strong blur to greatly simplify the outline (captures only the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # 3. Broad Canny thresholds to find the edge of the highly blurred shape
            low_threshold_outline = 10
            high_threshold_outline = 50 
            
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # 4. Dilate to thicken and ensure continuity
            kernel_outline = np.ones((5, 5), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=1) 
            
            # 5. Invert and combine
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the simple outline onto the final canvas
            final_line_drawing = np.minimum(final_line_drawing, inverted_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')


Starting selective line drawing for: divija.png
DEBUG: Saved ROI for eyes to: selective_line_drawing_eyes_roi.png
DEBUG: Saved ROI for nose to: selective_line_drawing_nose_roi.png
DEBUG: Saved ROI for mouth to: selective_line_drawing_mouth_roi.png
DEBUG: Saved ROI for philtrum to: selective_line_drawing_philtrum_roi.png
DEBUG: Saved ROI for eyebrows to: selective_line_drawing_eyebrows_roi.png
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760848373.956629 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760848373.957371 48300509 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760848373.961134 48300506 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760848373.986556 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760848373.987805 48300520 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [63]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe solutions (only Face Mesh is needed for visualization)
mp_face_mesh = mp.solutions.face_mesh


def visualize_all_landmarks(image_path, output_path):
    """
    Loads an image, processes it with MediaPipe Face Mesh, and draws colored 
    markers and their indices for ALL 468 detected landmarks.
    """
    print(f"Starting ALL landmark visualization for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Create a copy of the original image to draw on
    debug_image = img.copy()

    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results = face_mesh.process(img_rgb)
        
        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            
            # Iterate through ALL 468 landmarks
            for i, lm in enumerate(face_landmarks.landmark):
                # Convert normalized coordinates to pixel coordinates
                x = int(lm.x * W)
                y = int(lm.y * H)
                
                # Draw a small filled circle (landmark point) in white
                cv2.circle(debug_image, (x, y), 1, (255, 255, 255), -1)
                
                # Draw the index number in yellow for visibility
                cv2.putText(debug_image, str(i), (x + 2, y + 2), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)
                    
        else:
            print("Warning: Face landmarks not detected in the image.")

    # Save the result
    cv2.imwrite(output_path, debug_image)
    print(f"Full landmark visualization saved to: {output_path}")

# Example call
visualize_all_landmarks('divija.png', 'divija_all_features_debug.png')

Starting ALL landmark visualization for: divija.png


I0000 00:00:1760848369.921656 48225941 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760848369.923357 48300438 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760848369.927472 48300441 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/divija/Divi Drive/workplace/Princeton/Sem 5/Carlab/carlab_2025/final_project/carlab/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Full landmark visualization saved to: divija_all_features_debug.png


In [62]:
def preprocess_eyebrow_roi(input_path, output_path):
    """
    Loads the eyebrow ROI, converts it to grayscale, applies CLAHE for contrast, 
    and strong Gaussian blur for smoothing, preparing it for clean Canny detection.
    """
    # 1. Load the image
    img = cv2.imread(input_path)
    if img is None:
        print(f"Error: Could not read image at {input_path}. Ensure you have run the main script once to generate this file.")
        return

    print(f"Loaded image from: {input_path}")
    
    # 2. Convert to grayscale (required for CLAHE/Canny)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3. Apply CLAHE (Adaptive Contrast)
    # This boosts local contrast, helping separate the eyebrow outline from the skin.
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
    high_contrast_img = clahe.apply(gray_img) 

    # 4. Apply STRONG Gaussian Blur
    # The (31, 31) kernel size is used to destroy internal texture/noise, leaving
    # only the smooth, gross shape of the eyebrow for Canny to detect.
    blur_kernel = (61, 61)
    blurred_img = cv2.GaussianBlur(gray_img, blur_kernel, 0)
    equalized_img = cv2.equalizeHist(blurred_img)
    equalized_img = cv2.equalizeHist(equalized_img)

    edges = cv2.Canny(equalized_img, 0, 40)
    inverted_edges = cv2.bitwise_not(edges)
    # 5. Save the preprocessed image
    cv2.imwrite(output_path, inverted_edges)
    print(f"Preprocessed output saved to: {output_path}")


# --- Test Execution ---
# NOTE: This file must exist after running the main line drawing script!
INPUT_FILE = 'selective_line_drawing_eyebrows_roi.png'
OUTPUT_FILE = 'eyebrow_preprocessed_output.png'

preprocess_eyebrow_roi(INPUT_FILE, OUTPUT_FILE)


Loaded image from: selective_line_drawing_eyebrows_roi.png
Preprocessed output saved to: eyebrow_preprocessed_output.png


In [8]:
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Feature groups for distinct processing (Eyes, Nose, Mouth).
# Each group will be processed with its own tight bounding box.
FEATURE_GROUPS = {
    # Indices covering both eyes, excluding the face contour
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463, 224], 
    
    # INDICES UPDATED: Added points for better coverage of nostrils and the nose base.
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420, 129, 358, 64, 294, 98, 327, 118, 347], 
    
    # Indices covering the lips and corners of the mouth
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415],
    
    # UPDATED GROUP: Simplified indices to reliably anchor the region between nose (2) and lips (13, 14).
    # This creates a vertical bounding box for the philtrum.
    "philtrum": [2, 13, 14, 172, 406, ],
    
    # Indices covering both left and right eyebrows
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
    

}

# Define Canny thresholds based on feature importance/noise level
# Lower thresholds (e.g., 50, 150) capture more detail (high sensitivity)
# Higher thresholds (e.g., 80, 200) capture only strong, clean edges (low sensitivity/less noise)
CANNY_THRESHOLDS = {
    "eyes": (50, 180),       # High detail for eyes
    "nose": (50, 150),       # High detail for nose
    "mouth": (50, 180),      # High detail for lips
    "philtrum": (60, 150),   # Moderate detail for philtrum/creases
    "eyebrows": (10, 50),   # Low detail/Less noise for eyebrows
}
GAUSSIAN_BLUR_SIZES = {
    "eyes": (13, 13),
    "nose": (11, 11),
    "mouth": (11, 11),
    "philtrum": (31, 31),
    "eyebrows": (31, 31),
}

def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by applying CLAHE and Canny edge detection 
    independently to multiple small facial feature ROIs, then adding a minimal 
    outline for the hair and clothing.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Loop over each feature group to process them individually
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords = []
                y_coords = []
                
                # Calculate a tight bounding box *only* around the selected feature group
                for index in indices:
                    lm = face_landmarks.landmark[index]
                    x_coords.append(lm.x * W)
                    y_coords.append(lm.y * H)
                
                if not x_coords: continue # Skip if no landmarks are found
                
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                # Use a small fixed padding for these tiny ROIs
                padding = 15
                if feature_name == "nose" or feature_name == "philtrum":
                    # Increase padding for the nose and philtrum to prevent harsh cutoffs and ensure better blending
                    padding = 25 

                x1 = max(0, min_x - padding)
                y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding)
                y2 = min(H, max_y + padding)
                
                # Ensure ROI is valid
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the original image
                face_roi_area = img[y1:y2, x1:x2].copy()
                
                # --- DEBUG: Save the cropped ROI image for inspection ---
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_roi.png')
                cv2.imwrite(roi_debug_path, face_roi_area)
                print(f"DEBUG: Saved ROI for {feature_name} to: {roi_debug_path}")
                # --- END DEBUG ---

                face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
                
                # --- Selective CLAHE and Canny for High-Detail Features ---
                
                # Apply ADAPTIVE CONTRAST (CLAHE)
                clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                high_contrast_feature = clahe.apply(face_roi_gray)
                high_contrast_feature = cv2.equalizeHist(face_roi_gray)
                # Apply Gaussian Blur (kernel size adjusted slightly per feature type if needed, or stick to small (5, 5))
                blurred_feature = cv2.GaussianBlur(high_contrast_feature, GAUSSIAN_BLUR_SIZES.get(feature_name, (5, 5)), 0)
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_roi.png')
                cv2.imwrite(roi_debug_path, blurred_feature)
                print(f"DEBUG: Saved ROI for {feature_name} to: {roi_debug_path}")
                # --- END DYNAMIC CANNY THRESHOLDS ---
                # --- APPLY DYNAMIC CANNY THRESHOLDS ---
                low_thresh, high_thresh = CANNY_THRESHOLDS.get(feature_name, (50, 150))
                feature_edges = cv2.Canny(blurred_feature, low_thresh, high_thresh)

                
                
                # Dilate the feature lines slightly
                kernel_feature = np.ones((7, 7), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                # Invert the lines (black lines on white)
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)

                # Combine feature edges onto the final canvas (black lines (0) overwrite white (255))
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Simplified Mask Approach) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # --- SIMPLIFIED OUTER OUTLINE (Based on Mask Only) ---
            threshold = 0.5
            person_mask = results_segmentation.segmentation_mask > threshold
            
            # 1. Isolate the mask image (white person shape on black background)
            person_outline_img = person_mask.astype(np.uint8) * 255 

            # 2. Strong blur to greatly simplify the outline (captures only the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # 3. Broad Canny thresholds to find the edge of the highly blurred shape
            low_threshold_outline = 10
            high_threshold_outline = 50 
            
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # 4. Dilate to thicken and ensure continuity
            kernel_outline = np.ones((5, 5), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=1) 
            
            # 5. Invert and combine
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the simple outline onto the final canvas
            final_line_drawing = np.minimum(final_line_drawing, inverted_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')

Starting selective line drawing for: divija.png
DEBUG: Saved ROI for eyes to: selective_line_drawing_eyes_roi.png
DEBUG: Saved ROI for eyes to: selective_line_drawing_eyes_roi.png
DEBUG: Saved ROI for nose to: selective_line_drawing_nose_roi.png
DEBUG: Saved ROI for nose to: selective_line_drawing_nose_roi.png
DEBUG: Saved ROI for mouth to: selective_line_drawing_mouth_roi.png
DEBUG: Saved ROI for mouth to: selective_line_drawing_mouth_roi.png
DEBUG: Saved ROI for philtrum to: selective_line_drawing_philtrum_roi.png
DEBUG: Saved ROI for philtrum to: selective_line_drawing_philtrum_roi.png
DEBUG: Saved ROI for eyebrows to: selective_line_drawing_eyebrows_roi.png
DEBUG: Saved ROI for eyebrows to: selective_line_drawing_eyebrows_roi.png
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760851599.980152 48348156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760851599.980956 48356431 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760851599.984671 48356432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760851600.014032 48348156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760851600.014756 48356445 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [11]:
#GOOD CODE
# Initialize MediaPipe solutions (all required imports in one place)
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Feature groups for distinct processing (Eyes, Nose, Mouth).
# Each group will be processed with its own tight bounding box.
FEATURE_GROUPS = {
     # Indices covering both eyes, excluding the face contour
    # EXPANDED: Added points (e.g., 159, 386, 107, 336) near the brow and temple to increase the ROI size.
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463, 159, 386, 224], 
    
    # INDICES UPDATED: Added points for better coverage of nostrils and the nose base.
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420, 129, 358, 64, 294, 98, 327, 118, 347], 
    
    # Indices covering the lips and corners of the mouth
    # EXPANDED: Added points (e.g., 175, 404, 57, 287) near the chin and lower face to increase the ROI size.
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415, 199],

    
    # UPDATED GROUP: Simplified indices to reliably anchor the region between nose (2) and lips (13, 14).
    # This creates a vertical bounding box for the philtrum.
    "philtrum": [2, 13, 14, 172, 406, ],
    
    # Indices covering both left and right eyebrows
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
}

# Define Canny thresholds based on feature importance/noise level
CANNY_THRESHOLDS = {
    "eyes": (10, 100),       # High detail for eyes
    "nose": (0, 100),       # High detail for nose
    "mouth": (50, 120),      # High detail for lips
    "philtrum": (60, 120),   # Moderate detail for philtrum/creases
    "eyebrows": (10, 60),   # Low detail/Less noise for eyebrows
}
GAUSSIAN_BLUR_SIZES = {
    "eyes": (13, 13),
    "nose": (11, 11),
    "mouth": (11, 11),
    "philtrum": (31, 31),
    "eyebrows": (21, 21),
}

def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by applying CLAHE, LAPLACIAN, and Canny edge detection 
    independently to multiple small facial feature ROIs, then adding a minimal 
    outline for the hair and clothing.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Loop over each feature group to process them individually
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords = []
                y_coords = []
                
                # Calculate a tight bounding box *only* around the selected feature group
                for index in indices:
                    lm = face_landmarks.landmark[index]
                    x_coords.append(lm.x * W)
                    y_coords.append(lm.y * H)
                
                if not x_coords: continue # Skip if no landmarks are found
                
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                # Use a small fixed padding for these tiny ROIs
                padding = 15
                if feature_name == "nose" or feature_name == "philtrum":
                    # Increase padding for the nose and philtrum to prevent harsh cutoffs and ensure better blending
                    padding = 25 

                x1 = max(0, min_x - padding)
                y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding)
                y2 = min(H, max_y + padding)
                
                # Ensure ROI is valid
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the original image
                face_roi_area = img[y1:y2, x1:x2].copy()
                
               
                face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
                
                
                # apply histogram equalization
                high_contrast_feature = cv2.equalizeHist(face_roi_gray)
                
                # Apply Gaussian Blur (Size determined dynamically from GAUSSIAN_BLUR_SIZES)
                blur_size = GAUSSIAN_BLUR_SIZES.get(feature_name, (5, 5))
                blurred_feature = cv2.GaussianBlur(high_contrast_feature, blur_size, 0)
                clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                high_contrast_feature = clahe.apply(high_contrast_feature)
                #DEBUG: Save the blurred ROI for inspection
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_blurred.png')
                cv2.imwrite(roi_debug_path, blurred_feature)
                print(f"DEBUG: Saved blurred ROI for {feature_name} to: {roi_debug_path}")
                # --- END DEBUG ---
                # --- UNIVERSAL LAPLACIAN FILTERING ---
                # Apply Laplacian Filter (CV_64F for precision) to emphasize line features
                laplacian = cv2.Laplacian(blurred_feature, cv2.CV_64F)
                # Convert to 8-bit for Canny input (using absolute values)
                canny_input = cv2.convertScaleAbs(laplacian)
                print(f"INFO: Using Laplacian filter output for {feature_name} Canny input.")
                # --- END UNIVERSAL LAPLACIAN FILTERING ---
                
                # --- APPLY DYNAMIC CANNY THRESHOLDS ---
                low_thresh, high_thresh = CANNY_THRESHOLDS.get(feature_name, (50, 150))
                
                # Canny now uses the Laplacian output
                feature_edges = cv2.Canny(blurred_feature, low_thresh, high_thresh)

                
                # Dilate the feature lines slightly (Kernel size (3,3) with iteration 1 is often cleaner than 7x7)
                kernel_feature = np.ones((7, 7), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                # Invert the lines (black lines on white)
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)
                #debug: Save the inverted edges for inspection
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_inverted_edges.png')
                cv2.imwrite(roi_debug_path, inverted_feature_edges)
                print(f"DEBUG: Saved inverted edges for {feature_name} to: {roi_debug_path}")
                # Combine feature edges onto the final canvas (black lines (0) overwrite white (255))
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Simplified Mask Approach) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # --- SIMPLIFIED OUTER OUTLINE (Based on Mask Only) ---
            threshold = 0.5
            person_mask = results_segmentation.segmentation_mask > threshold
            
            # 1. Isolate the mask image (white person shape on black background)
            person_outline_img = person_mask.astype(np.uint8) * 255 

            # 2. Strong blur to greatly simplify the outline (captures only the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # 3. Broad Canny thresholds to find the edge of the highly blurred shape
            low_threshold_outline = 10
            high_threshold_outline = 50 
            
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # 4. Dilate to thicken and ensure continuity
            kernel_outline = np.ones((5, 5), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=1) 
            
            # 5. Invert and combine
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the simple outline onto the final canvas
            final_line_drawing = np.minimum(final_line_drawing, inverted_outline)

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')


Starting selective line drawing for: divija.png
DEBUG: Saved blurred ROI for eyes to: selective_line_drawing_eyes_blurred.png
INFO: Using Laplacian filter output for eyes Canny input.
DEBUG: Saved inverted edges for eyes to: selective_line_drawing_eyes_inverted_edges.png
DEBUG: Saved blurred ROI for nose to: selective_line_drawing_nose_blurred.png
INFO: Using Laplacian filter output for nose Canny input.
DEBUG: Saved inverted edges for nose to: selective_line_drawing_nose_inverted_edges.png
DEBUG: Saved blurred ROI for mouth to: selective_line_drawing_mouth_blurred.png
INFO: Using Laplacian filter output for mouth Canny input.
DEBUG: Saved inverted edges for mouth to: selective_line_drawing_mouth_inverted_edges.png
DEBUG: Saved blurred ROI for philtrum to: selective_line_drawing_philtrum_blurred.png
INFO: Using Laplacian filter output for philtrum Canny input.
DEBUG: Saved inverted edges for philtrum to: selective_line_drawing_philtrum_inverted_edges.png
DEBUG: Saved blurred ROI for ey

I0000 00:00:1760851716.763782 48348156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760851716.764531 48358285 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760851716.768074 48358284 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760851716.787882 48348156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760851716.788642 48358302 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# --- 1. Catmull-Rom Spline Interpolation Function ---
# (Keep this function as is, it's generally useful for all smooth lines)
def catmull_rom_spline(P0, P1, P2, P3, num_points=20):
    P0, P1, P2, P3 = np.array(P0), np.array(P1), np.array(P2), np.array(P3)
    
    points = []
    for t in np.linspace(0, 1, num_points, endpoint=False):
        t2 = t * t
        t3 = t2 * t
        
        h00 = 2 * t3 - 3 * t2 + 1
        h10 = t3 - 2 * t2 + t
        h01 = -2 * t3 + 3 * t2
        h11 = t3 - t2
        
        M1 = 0.5 * (P2 - P0)
        M2 = 0.5 * (P3 - P1)
        
        P_t = h00 * P1 + h10 * M1 + h01 * P2 + h11 * M2
        points.append(P_t.astype(int))
    
    return points

def draw_smooth_curve(image, points, color=(0, 0, 0), thickness=2, is_closed=False):
    if len(points) < 2:
        return

    extended_points = list(points)
    if is_closed:
        extended_points.append(points[0]) 
        extended_points.insert(0, points[-1])
        extended_points.append(points[1])
    else:
        extended_points.insert(0, points[0])
        extended_points.append(points[-1])
        
    all_interpolated_points = []
    
    for i in range(1, len(extended_points) - 2):
        P0 = extended_points[i-1]
        P1 = extended_points[i]
        P2 = extended_points[i+1]
        P3 = extended_points[i+2]
        
        segment_points = catmull_rom_spline(P0, P1, P2, P3, num_points=10)
        all_interpolated_points.extend(segment_points)

    all_interpolated_points.append(points[-1])

    for i in range(len(all_interpolated_points) - 1):
        pt1 = tuple(all_interpolated_points[i])
        pt2 = tuple(all_interpolated_points[i+1])
        cv2.line(image, pt1, pt2, color, thickness, cv2.LINE_AA)
        
    if is_closed:
        cv2.line(image, tuple(points[-1]), tuple(points[0]), color, thickness, cv2.LINE_AA)

# --- 2. Landmark Indices for Features ---
LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398] # Outer ring
RIGHT_EYE = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7] # Outer ring
LIPS_OUTER = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 306, 375, 321, 405, 314, 17]
LIPS_INNER = [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308, 309, 310, 311, 415, 307] 
FACE_OVAL = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109]

# Pupil center landmarks (Refine_landmarks=True adds these)
LEFT_PUPIL_CENTER = 468
RIGHT_PUPIL_CENTER = 473

# --- 3. Main Processing Logic ---

# IMPORTANT: Replace 'input_image.jpg' with a path to your face image
image_path = 'divija.p' 

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True, 
    max_num_faces=1,
    refine_landmarks=True, 
    min_detection_confidence=0.5
)

image = cv2.imread(image_path)
if image is None:
    print(f"Error: Could not load image at {image_path}")
    exit()

canvas = np.ones_like(image) * 255
h, w, _ = image.shape

results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

if results.multi_face_landmarks:
    face_landmarks = results.multi_face_landmarks[0]
    
    pixel_landmarks = {}
    for i, landmark in enumerate(face_landmarks.landmark):
        x = int(landmark.x * w)
        y = int(landmark.y * h)
        pixel_landmarks[i] = (x, y)

    def get_contour_points(indices):
        return [pixel_landmarks[i] for i in indices if i in pixel_landmarks]

    line_color = (0, 0, 0) 
    line_thickness = 2
    
    # Draw Lips
    outer_lip_points = get_contour_points(LIPS_OUTER)
    draw_smooth_curve(canvas, outer_lip_points, line_color, line_thickness, is_closed=True)
    inner_lip_points = get_contour_points(LIPS_INNER)
    draw_smooth_curve(canvas, inner_lip_points, line_color, line_thickness, is_closed=True)
    
    # Draw Eyes
    left_eye_points = get_contour_points(LEFT_EYE)
    draw_smooth_curve(canvas, left_eye_points, line_color, line_thickness, is_closed=True)
    right_eye_points = get_contour_points(RIGHT_EYE)
    draw_smooth_curve(canvas, right_eye_points, line_color, line_thickness, is_closed=True)

    # Draw Face Outline
    face_oval_points = get_contour_points(FACE_OVAL)
    draw_smooth_curve(canvas, face_oval_points, line_color, line_thickness, is_closed=True)

    # --- MODIFIED: Draw Pupil Lines ---
    pupil_line_thickness = 3 # Make pupil line thicker for emphasis
    
    # Right Eye Pupil
    if RIGHT_PUPIL_CENTER in pixel_landmarks:
        pupil_center_right = pixel_landmarks[RIGHT_PUPIL_CENTER]
        
        # Check if target landmarks exist and draw lines
        if 145 in pixel_landmarks:
            cv2.line(canvas, pupil_center_right, pixel_landmarks[145], line_color, pupil_line_thickness, cv2.LINE_AA)
        if 159 in pixel_landmarks:
            cv2.line(canvas, pupil_center_right, pixel_landmarks[159], line_color, pupil_line_thickness, cv2.LINE_AA)

    # Left Eye Pupil (Using analogous points for the left eye)
    # Based on MediaPipe's structure, 374 and 380 are analogous to 145 and 159 respectively
    if LEFT_PUPIL_CENTER in pixel_landmarks:
        pupil_center_left = pixel_landmarks[LEFT_PUPIL_CENTER]
        
        if 374 in pixel_landmarks: # Analogous to 145
            cv2.line(canvas, pupil_center_left, pixel_landmarks[374], line_color, pupil_line_thickness, cv2.LINE_AA)
        if 380 in pixel_landmarks: # Analogous to 159
            cv2.line(canvas, pupil_center_left, pixel_landmarks[380], line_color, pupil_line_thickness, cv2.LINE_AA)

# --- 4. Display Result ---
cv2.imwrite('stylized_line_drawing_with_custom_pupils.png', canvas)
print("Saved stylized line drawing with custom pupils to 'stylized_line_drawing_with_custom_pupils.png'")

Error: Could not load image at divija.p


I0000 00:00:1760851117.156375 48347869 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
[ WARN:0@0.330] global loadsave.cpp:275 findDecoder imread_('divija.p'): can't open/read file: check file path/integrity
W0000 00:00:1760851117.157261 48347994 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760851117.160887 48347993 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


AttributeError: 'NoneType' object has no attribute 'shape'

: 

In [6]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe solutions (all required imports in one place)
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# Feature groups and thresholds remain the same
FEATURE_GROUPS = {
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463, 159, 386, 224], 
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420, 129, 358, 64, 294, 98, 327, 118, 347], 
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415, 199],
    "philtrum": [2, 13, 14, 172, 406, ],
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
}
CANNY_THRESHOLDS = {
    "eyes": (10, 100), "nose": (5, 100), "mouth": (50, 100),
    "philtrum": (60, 120), "eyebrows": (10, 30),
}
GAUSSIAN_BLUR_SIZES = {
    "eyes": (13, 13), "nose": (11, 11), "mouth": (11, 11),
    "philtrum": (31, 31), "eyebrows": (41, 41),
}


def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and Edge Detection ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # --- Pre-convert the full image to Lab color space once ---
            img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
            
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords, y_coords = [], []
                
                # Bounding Box Calculation (omitted for brevity, assume correct)
                for index in indices:
                    if index < len(face_landmarks.landmark):
                         lm = face_landmarks.landmark[index]
                         x_coords.append(lm.x * W); y_coords.append(lm.y * H)
                
                if not x_coords: continue 
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                padding = 30
                if feature_name in ["nose", "philtrum"]: padding = 25 
                
                x1 = max(0, min_x - padding); y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding); y2 = min(H, max_y + padding)
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the Lab image
                face_roi_lab = img_lab[y1:y2, x1:x2].copy()
                
                # --- COLOR DIFFERENCE EDGE EXTRACTION ---
                all_channel_laplacian_edges = []
                blur_size = GAUSSIAN_BLUR_SIZES.get(feature_name, (5, 5))
                
                # Iterate over the L, a, and b channels
                for channel_index in range(3):
                    channel = face_roi_lab[:, :, channel_index]
                    
                    # 1. Apply CLAHE only to the L channel (luminosity)
                    if channel_index == 0:
                        channel = clahe.apply(channel)
                        
                    # 2. Apply Gaussian Blur
                    blurred_channel = cv2.GaussianBlur(channel, blur_size, 0)
                    
                    # 3. Apply Laplacian (Color Difference / Lightness Difference)
                    laplacian = cv2.Laplacian(blurred_channel, cv2.CV_64F, ksize=3)
                    
                    # 4. Convert to 8-bit absolute value and normalize the edge response
                    laplacian_abs = cv2.convertScaleAbs(laplacian)
                    
                    all_channel_laplacian_edges.append(laplacian_abs)

                # --- Combine Edges from L, a, and b channels ---
                # A simple weighted sum is effective: L (lightness) often needs to be balanced 
                # with a/b (color). We'll use the maximum response for robust edge capture.
                canny_input = cv2.max(all_channel_laplacian_edges[0], all_channel_laplacian_edges[1])
                canny_input = cv2.max(canny_input, all_channel_laplacian_edges[2])
                
                
                # --- APPLY DYNAMIC CANNY THRESHOLDS ---
                low_thresh, high_thresh = CANNY_THRESHOLDS.get(feature_name, (50, 150))
                
                # Canny is applied to the combined, normalized Laplacian map
                feature_edges = cv2.Canny(canny_input, low_thresh, high_thresh)

                # --- Post-processing: Dilate and Combine ---
                kernel_feature = np.ones((3, 3), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)
                
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)
                print(f"Processed and merged {feature_name} edges using Lab color differences.")

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Unchanged) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            threshold = 0.5
            person_mask = results_segmentation.segmentation_mask > threshold
            person_outline_img = person_mask.astype(np.uint8) * 255 

            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)
            outline_edges = cv2.Canny(blurred_outline, 10, 50)
            
            kernel_outline = np.ones((5, 5), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=1) 
            
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            final_line_drawing = np.minimum(final_line_drawing, inverted_outline)
            print("Processed and merged outer outline edges.")

    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija.png', 'selective_line_drawing.png')

Starting selective line drawing for: divija.png
Processed and merged eyes edges using Lab color differences.
Processed and merged nose edges using Lab color differences.
Processed and merged mouth edges using Lab color differences.
Processed and merged philtrum edges using Lab color differences.
Processed and merged eyebrows edges using Lab color differences.
Processed and merged outer outline edges.
Final selective line drawing saved to: selective_line_drawing.png


I0000 00:00:1760851483.164439 48348156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760851483.165217 48354507 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760851483.168703 48354506 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760851483.259895 48348156 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760851483.260592 48354520 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [41]:
import cv2
import numpy as np
from skimage.morphology import skeletonize 

def simplify_for_laser_cutter_merged(image_path, epsilon_factor=0.005, closing_kernel_size=3, min_contour_length=50, output_canvas_size=(800, 800), output_line_thickness=2):
    """
    Produces a single-line drawing by applying morphological closing to merge nearby parallel lines, 
    followed by skeletonization and RDP simplification.

    Args:
        image_path (str): Path to the input line art image.
        epsilon_factor (float): RDP simplification factor (Higher = smoother/simpler).
        closing_kernel_size (int): Size of the kernel for the CLOSE operation. Increase this 
                                   to merge lines that are further apart. Odd numbers (e.g., 3, 5, 7) are best.
        min_contour_length (int): Filters out short lines/noise.
        output_canvas_size (tuple): Output image size.
        output_line_thickness (int): Thickness for the output visualization image.

    Returns:
        np.ndarray: A clean, simplified, single-line drawing.
        list: The list of simplified polygonal contours (vector paths).
    """
    print(f"Starting single-line vector simplification for: {image_path}")
    
    # --- 1. Load and Binarize ---
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Invert and Binarize: Lines are white (255) on black (0) for morphological ops
    _, binary_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    
    # --- 2. Morphological Closing (MERGING STEP) ---
    # This closes small holes and joins very close, parallel lines into a single blob.
    kernel = np.ones((closing_kernel_size, closing_kernel_size), np.uint8)
    # Closing = Dilate (grow) then Erode (shrink). This fills small gaps and connects nearby lines.
    merged_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)
    print(f"Applied morphological closing with kernel size: {closing_kernel_size}.")
    
    # --- 3. Skeletonize ---
    # Convert to boolean for skeletonize, then reduce all merged lines to 1-pixel thickness
    merged_bool = merged_img / 255.0 > 0.5 
    skeleton = skeletonize(merged_bool)
    skeleton_img = (skeleton * 255).astype(np.uint8) 
    
    # --- 4. Contour Tracing and Simplification (RDP) ---
    contours, _ = cv2.findContours(skeleton_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    simplified_contours = []
    output_canvas = np.ones((output_canvas_size[1], output_canvas_size[0], 3), dtype=np.uint8) * 255
    
    # Scaling and Offset calculations
    img_h, img_w = img.shape[:2]
    scale_factor = min(output_canvas_size[0] / img_w, output_canvas_size[1] / img_h)
    offset_x = (output_canvas_size[0] - img_w * scale_factor) / 2
    offset_y = (output_canvas_size[1] - img_h * scale_factor) / 2

    for contour in contours:
        if len(contour) < 2: continue
        
        perimeter = cv2.arcLength(contour, False) 
        if perimeter < min_contour_length: continue

        # Ramer-Douglas-Peucker Simplification
        epsilon = epsilon_factor * perimeter
        approx = cv2.approxPolyDP(contour, epsilon, False)
        
        if len(approx) < 2: continue
            
        # Apply Scaling and store
        scaled_approx = np.array([[[int(p[0][0] * scale_factor + offset_x), int(p[0][1] * scale_factor + offset_y)]] for p in approx], dtype=np.int32)
        simplified_contours.append(scaled_approx)
        
        # Draw the single-line path
        cv2.drawContours(output_canvas, [scaled_approx], -1, (0, 0, 0), output_line_thickness, cv2.LINE_AA)
        
    print(f"Final paths after merging and simplification: {len(simplified_contours)}.")
    return output_canvas, simplified_contours

# --- Example Usage ---

# Use the image you provided.
input_image_path = 'selective_line_drawing2.png' 

try:
    output_image, final_vectors = simplify_for_laser_cutter_merged(
        input_image_path, 
        epsilon_factor=0.0001,          # Increased smoothing factor
        closing_kernel_size=5,         # Increased to 5 to merge lines slightly further apart
        min_contour_length=75,         # Filter out small artifacts
        output_line_thickness=1
    )

    
    cv2.imwrite('merged_single_line_output.png', output_image)
    

except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"An error occurred: {e}")

Starting single-line vector simplification for: selective_line_drawing2.png
Applied morphological closing with kernel size: 5.
Final paths after merging and simplification: 58.


In [40]:

import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
# Initialize MediaPipe solutions (all required imports in one place)
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation
HAIR_MODEL_PATH = '/Users/divija/Divi Drive/workplace/Princeton/Sem 5/Carlab/carlab_2025/final_project/hair_segmenter.tflite'  # Path to custom hair segmentation model
# Feature groups for distinct processing (Eyes, Nose, Mouth).
# Each group will be processed with its own tight bounding box.
FEATURE_GROUPS = {
     # Indices covering both eyes, excluding the face contour
    # EXPANDED: Added points (e.g., 159, 386, 107, 336) near the brow and temple to increase the ROI size.
    "eyes": [33, 133, 163, 144, 246, 362, 263, 390, 373, 466, 130, 160, 243, 359, 387, 463, 159, 386, 224], 
    
    # INDICES UPDATED: Added points for better coverage of nostrils and the nose base.
    "nose": [1, 6, 197, 2, 4, 5, 278, 275, 420, 129, 358, 64, 294, 98, 327, 118, 347], 
    
    # Indices covering the lips and corners of the mouth
    # EXPANDED: Added points (e.g., 175, 404, 57, 287) near the chin and lower face to increase the ROI size.
    "mouth": [0, 17, 61, 291, 14, 308, 375, 40, 81, 311, 317, 415, 199],

    
    # UPDATED GROUP: Simplified indices to reliably anchor the region between nose (2) and lips (13, 14).
    # This creates a vertical bounding box for the philtrum.
    "philtrum": [2, 13, 14, 172, 406, ],
    
    # Indices covering both left and right eyebrows
    "eyebrows": [46, 53, 52, 65, 55, 70, 63, 105, 107, 276, 283, 282, 295, 285, 336, 296, 334, 293, 300],
}

# Define Canny thresholds based on feature importance/noise level
CANNY_THRESHOLDS = {
    "eyes": (10, 100),       # High detail for eyes
    "nose": (0, 70),       # High detail for nose
    "mouth": (50, 120),      # High detail for lips
    "philtrum": (60, 120),   # Moderate detail for philtrum/creases
    "eyebrows": (10, 60),   # Low detail/Less noise for eyebrows
}
GAUSSIAN_BLUR_SIZES = {
    "eyes": (13, 13),
    "nose": (11, 11),
    "mouth": (11, 11),
    "philtrum": (31, 31),
    "eyebrows": (21, 21),
}

def image_to_minimal_line_drawing_enhanced(image_path, output_path):
    """
    Creates a highly stylized line drawing by applying CLAHE, LAPLACIAN, and Canny edge detection 
    independently to multiple small facial feature ROIs, then adding a minimal 
    outline for the hair and clothing.
    """
    print(f"Starting selective line drawing for: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the final line drawing canvas: A white background (255)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255 
    
    # --- 1. Face Mesh for Multiple Feature ROIs and CLAHE ---
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, 
        max_num_faces=1, 
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        results_face = face_mesh.process(img_rgb)
        
        if results_face.multi_face_landmarks:
            
            # Use the first detected face
            face_landmarks = results_face.multi_face_landmarks[0]
            
            # Loop over each feature group to process them individually
            for feature_name, indices in FEATURE_GROUPS.items():
                x_coords = []
                y_coords = []
                
                # Calculate a tight bounding box *only* around the selected feature group
                for index in indices:
                    lm = face_landmarks.landmark[index]
                    x_coords.append(lm.x * W)
                    y_coords.append(lm.y * H)
                
                if not x_coords: continue # Skip if no landmarks are found
                
                min_x, max_x = int(min(x_coords)), int(max(x_coords))
                min_y, max_y = int(min(y_coords)), int(max(y_coords))

                # Use a small fixed padding for these tiny ROIs
                padding = 15
                if feature_name == "nose" or feature_name == "philtrum":
                    # Increase padding for the nose and philtrum to prevent harsh cutoffs and ensure better blending
                    padding = 25 

                x1 = max(0, min_x - padding)
                y1 = max(0, min_y - padding)
                x2 = min(W, max_x + padding)
                y2 = min(H, max_y + padding)
                
                # Ensure ROI is valid
                if x2 <= x1 or y2 <= y1: continue

                # Extract the ROI from the original image
                face_roi_area = img[y1:y2, x1:x2].copy()
                
                face_roi_area = cv2.medianBlur(face_roi_area, 11)
                face_roi_gray = cv2.cvtColor(face_roi_area, cv2.COLOR_BGR2GRAY)
                
                
                # apply histogram equalization
                high_contrast_feature = cv2.equalizeHist(face_roi_gray)
                
                # Apply Gaussian Blur (Size determined dynamically from GAUSSIAN_BLUR_SIZES)
                blur_size = GAUSSIAN_BLUR_SIZES.get(feature_name, (5, 5))
                blurred_feature = cv2.GaussianBlur(high_contrast_feature, blur_size, 0)
                clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                high_contrast_feature = clahe.apply(high_contrast_feature)
                #DEBUG: Save the blurred ROI for inspection
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_blurred.png')
                cv2.imwrite(roi_debug_path, blurred_feature)
                print(f"DEBUG: Saved blurred ROI for {feature_name} to: {roi_debug_path}")
                # --- END DEBUG ---
                # --- UNIVERSAL LAPLACIAN FILTERING ---
                # Apply Laplacian Filter (CV_64F for precision) to emphasize line features
                laplacian = cv2.Laplacian(blurred_feature, cv2.CV_64F)
                # Convert to 8-bit for Canny input (using absolute values)
                canny_input = cv2.convertScaleAbs(laplacian)
                print(f"INFO: Using Laplacian filter output for {feature_name} Canny input.")
                # --- END UNIVERSAL LAPLACIAN FILTERING ---
                
                # --- APPLY DYNAMIC CANNY THRESHOLDS ---
                low_thresh, high_thresh = CANNY_THRESHOLDS.get(feature_name, (50, 150))
                
                # Canny now uses the Laplacian output
                feature_edges = cv2.Canny(blurred_feature, low_thresh, high_thresh)

                
                # Dilate the feature lines slightly (Kernel size (3,3) with iteration 1 is often cleaner than 7x7)
                kernel_feature = np.ones((7, 7), np.uint8) 
                dilated_feature_edges = cv2.dilate(feature_edges, kernel_feature, iterations=1) 
                
                # Invert the lines (black lines on white)
                inverted_feature_edges = cv2.bitwise_not(dilated_feature_edges)
                #debug: Save the inverted edges for inspection
                roi_debug_path = output_path.replace('.png', f'_{feature_name}_inverted_edges.png')
                cv2.imwrite(roi_debug_path, inverted_feature_edges)
                print(f"DEBUG: Saved inverted edges for {feature_name} to: {roi_debug_path}")
                # Combine feature edges onto the final canvas (black lines (0) overwrite white (255))
                current_canvas_roi = final_line_drawing[y1:y2, x1:x2]
                final_line_drawing[y1:y2, x1:x2] = np.minimum(current_canvas_roi, inverted_feature_edges)

    # --- 2. Selfie Segmentation for Hair and Clothing Outline (Simplified Mask Approach) ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        
        if results_segmentation.segmentation_mask is not None:
            
            # --- SIMPLIFIED OUTER OUTLINE (Based on Mask Only) ---
            threshold = 0.5
            person_mask = results_segmentation.segmentation_mask > threshold
            
            # 1. Isolate the mask image (white person shape on black background)
            person_outline_img = person_mask.astype(np.uint8) * 255 

            # 2. Strong blur to greatly simplify the outline (captures only the gross shape)
            blurred_outline = cv2.GaussianBlur(person_outline_img, (61, 61), 0)

            # 3. Broad Canny thresholds to find the edge of the highly blurred shape
            low_threshold_outline = 10
            high_threshold_outline = 50 
            
            outline_edges = cv2.Canny(blurred_outline, low_threshold_outline, high_threshold_outline)
            
            # 4. Dilate to thicken and ensure continuity
            kernel_outline = np.ones((5, 5), np.uint8) 
            dilated_outline_edges = cv2.dilate(outline_edges, kernel_outline, iterations=1) 
            
            # 5. Invert and combine
            inverted_outline = cv2.bitwise_not(dilated_outline_edges)
            
            # Combine the simple outline onto the final canvas
            final_line_drawing = np.minimum(final_line_drawing, inverted_outline)
    if HAIR_MODEL_PATH:
        print("Starting dedicated hair segmentation.")
        try:
            base_options = python.BaseOptions(model_asset_path=HAIR_MODEL_PATH, delegate=None)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)
            options = vision.ImageSegmenterOptions(
                base_options=base_options,
                running_mode=vision.RunningMode.IMAGE,
                output_category_mask=True  # Hair is category 1
            )
            with vision.ImageSegmenter.create_from_options(options) as segmenter:
                segmentation_result = segmenter.segment(mp_image)
                category_mask = segmentation_result.category_mask.numpy_view()

                # Hair mask (category index 1)
                hair_mask = np.where(category_mask == 1, 255, 0).astype(np.uint8)
                
                # Resize the mask to the original image dimensions
                resized_mask = cv2.resize(hair_mask, (W, H), interpolation=cv2.INTER_NEAREST)
                
                # 1. Blur the mask for a smoother outline
                blurred_hair_mask = cv2.GaussianBlur(resized_mask, (21, 21), 0)
                
                # 2. Canny edge detection on the smoothed mask
                hair_outline_edges = cv2.Canny(blurred_hair_mask, 10, 50)
                
                # 3. Dilate and Invert
                kernel_hair = np.ones((5, 5), np.uint8) 
                dilated_hair_edges = cv2.dilate(hair_outline_edges, kernel_hair, iterations=1) 
                inverted_hair_outline = cv2.bitwise_not(dilated_hair_edges)
                
                # Combine the hair outline
                final_line_drawing = np.minimum(final_line_drawing, inverted_hair_outline)
                print("Processed and merged dedicated hair outline.")

        except Exception as e:
            print(f"ERROR in Hair Segmentation: Ensure '{HAIR_MODEL_PATH}' is correct. Falling back to Selfie Segmentation for hair/body.")
    # Save the result
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Final selective line drawing saved to: {output_path}")

# Example call for normal execution
image_to_minimal_line_drawing_enhanced('divija2.png', 'selective_line_drawing2.png')

Starting selective line drawing for: divija2.png
DEBUG: Saved blurred ROI for eyes to: selective_line_drawing2_eyes_blurred.png
INFO: Using Laplacian filter output for eyes Canny input.
DEBUG: Saved inverted edges for eyes to: selective_line_drawing2_eyes_inverted_edges.png
DEBUG: Saved blurred ROI for nose to: selective_line_drawing2_nose_blurred.png
INFO: Using Laplacian filter output for nose Canny input.
DEBUG: Saved inverted edges for nose to: selective_line_drawing2_nose_inverted_edges.png


I0000 00:00:1760854637.812722 48376247 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760854637.813421 48417125 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760854637.817228 48417124 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760854637.906366 48376247 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760854637.907291 48417137 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760854637.953984 48376247 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760854637.955265 48417149 inference_feedback_manager.cc:114] Feedback manager requires 

DEBUG: Saved blurred ROI for mouth to: selective_line_drawing2_mouth_blurred.png
INFO: Using Laplacian filter output for mouth Canny input.
DEBUG: Saved inverted edges for mouth to: selective_line_drawing2_mouth_inverted_edges.png
DEBUG: Saved blurred ROI for philtrum to: selective_line_drawing2_philtrum_blurred.png
INFO: Using Laplacian filter output for philtrum Canny input.
DEBUG: Saved inverted edges for philtrum to: selective_line_drawing2_philtrum_inverted_edges.png
DEBUG: Saved blurred ROI for eyebrows to: selective_line_drawing2_eyebrows_blurred.png
INFO: Using Laplacian filter output for eyebrows Canny input.
DEBUG: Saved inverted edges for eyebrows to: selective_line_drawing2_eyebrows_inverted_edges.png
Starting dedicated hair segmentation.
Processed and merged dedicated hair outline.
Final selective line drawing saved to: selective_line_drawing2.png
